In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import os
import requests
import subprocess
import pandas as pd
import re
import glob
import csv

# 로우 데이터 확보를 위한 작업

1. hwp 파일 다운로드
2. csv 파일 변환
3. 한자 -> 한글 변환
4. 모든 csv 파일을 병합하여 df 만들기
---
5. 사람별로 컬럼 나누기 (◯를 기준으로 발언자가 나뉨)
6. 국회에 기록된 국회의원의 발언만 정리하기(국회의원 db 확보 완료)
7. 전체 국회 회기에 대해 국회의원 이름/정당(좌우)/발언으로 나뉘어진 df 생성

# csv 폴더 불러와서 파일 병합

In [36]:
# 현재 디렉토리에 있는 모든 csv 파일의 이름을 가져옵니다.
csv_files = glob.glob('D:\\MJ 데이터 공모전 로우 데이터\\국회의원 속기록(csv)\\16대_csv\\*.csv')

# 각 파일에서 첫 번째 행을 추출하여 저장할 빈 리스트를 생성합니다.
data_rows = []

# 각 파일에 대해 반복
for file_name in csv_files:
    # csv 파일을 읽습니다.
    # 파일에 헤더가 없음을 알리기 위해 header=None을 추가합니다.
    # 또한, 따옴표를 무시하도록 quoting=csv.QUOTE_NONE을 추가합니다.
    try:
        df = pd.read_csv(file_name, header=None, quoting=csv.QUOTE_NONE)
    except pd.errors.ParserError:
        print(f'Error parsing file {file_name}')
        continue

    # csv 파일이 비어있는지 확인합니다.
    if df.empty:
        print(f'{file_name} is empty')
        continue

    # 첫 번째 행만 추출하여 data_rows에 추가합니다.
    # 이때, 파일 이름을 첫 번째 열에 추가합니다.
    first_row = df.iloc[0].copy()  # 첫 번째 행이 데이터이므로 이를 사용합니다.
    first_row = first_row.tolist()  # 리스트로 변환합니다.
    first_row.insert(0, os.path.basename(file_name))  # 리스트의 첫 번째 위치에 파일 이름을 삽입합니다.
    data_rows.append(first_row)

# 모든 첫 번째 행을 하나의 데이터프레임으로 결합합니다.
merged_df = pd.DataFrame(data_rows)

In [37]:
merged_df['merged'] = merged_df.apply(lambda row: ' '.join(row.dropna().values.astype(str)), axis=1)
origin_df = merged_df[['merged']]

In [85]:
origin_df = merged_df.apply(lambda row: ' '.join(row.dropna().astype(str)), axis=1).to_frame()

In [87]:
origin_df.iloc[0,0]

'국회회의록_16대_212회_1차_건설교통위원회.csv 0<표>(제212회―건설교통제1차) <그림>(제212회―건설교통제1차)<그림> (14시13분 개의)○위원장 금영일 의석을 정돈해 주시기 바랍니다. 성원이 되었으므로 제212회 국회 임시회 제1차 건설교통위원회를 개의하겠습니다. 여러 위원님들 정말 반갑습니다. 의사일정에 앞서서 간단한 저의 인사말씀부터 올리도록 하겠습니다.  먼저 여러모로 부족한 저를 위원장으로 선출해 주신 여러 위원님들께 이 자리를 빌려서 다시 한번 감사의 말씀을 드립니다. 평소 제가 존경하는 훌륭하신 여러 선배님들과 동료위원 여러분들을 모시고 당면한 우리 국가경쟁력 제고와 국민들의 삶의 질 향상과 직결되는 건설교통분야에 대한 국정을 다루는 건설교통위원회에서 함께 일하게 된 것을 무한한 영광으로 생각합니다. 여러 가지로 부족하고 부덕한 제가 막중한 위원장의 직무를 잘 해낼 수 있을지 정말 걱정과 두려움이 앞섭니다마는 많은 경륜과 식견을 가지신 여러 위원님들의 지도편달을 받고 또 위원회를 운영함에 있어서 여러 위원님들과 항상 함께 호흡하고 또한 공인으로서 항상 국민의 편에 서서 모든 일을 공정하게 처리해 나간다면 맡은 바 소임을 다할 수 있지 않을까 감히 생각을 해봅니다. 제16대 국회는 새천년이 시작되는 원년에 임기가 개시되는 역사적인 시점의 국회인 만큼 우리 국회에 거는 국민의 기대는 매우 크다 하겠습니다." 21세기 지식정보화시대는 모든 분야에서 종래에는 상상조차 할 수 없었던 엄청난 변화와 도전이 예상되고 있습니다. 지금까지는 거의 국내시장에만 안주해 왔던 우리 건설산업도 이제는 국적과 관계없이 전세계의 모든 기업과 공격적인 경쟁을 해야 하고 또 절대량을 늘리는 데만 급급해 왔던 주택과 도로  공항 등 SOC시설도 이제는 그 질도 함께 고려해야만 할 것이며 지역간 균형있는 배분을 바라는 국민들의 요구 또한 이를 최대한 조화롭게 수용해 나가야 할 것입니다." 이러한 변화와 도전에 슬기롭게 대처하기 위해서는 무엇보다 우리 위원회의 운영을

## 여기에 각 회기별 특징적인 예외 단어들을 추가하면 됩니다.

In [88]:
# 17대에서 각 의원별 발언을 구분하는 ◯와 ○가 구분없이 사용되었으므로 수정했습니다.
origin_df = origin_df.applymap(lambda x: str(x).replace('○', '◯') if pd.notnull(x) else x)
origin_df = origin_df.applymap(lambda x: str(x).replace('○', '◯') if pd.notnull(x) else x)

In [89]:
# 16대에는 'ꠏ', 'ꠚ'(서로 다름)라는 특수문자가 의미없이 출현하므로 삭제했습니다. 읽어본 결과 특수문자 or 기호를 표현한 것 같습니다.
origin_df = origin_df.applymap(lambda x: str(x).replace('ꠏ', '') if pd.notnull(x) else x)
origin_df = origin_df.applymap(lambda x: str(x).replace('ꠚ', '') if pd.notnull(x) else x)

In [90]:
# 16대 파일에서 의미없는 단어를 찾아 삭제했습니다.
origin_df = origin_df.applymap(lambda x: str(x).replace('\u3000', ' ') if pd.notnull(x) else x)

In [91]:
# 한자 -> 한글 변환 과정에서 문제가 있는 단어를 변경했습니다.
origin_df = origin_df.applymap(lambda x: str(x).replace('립법', '입법') if pd.notnull(x) else x)

In [92]:
# 모든 회기에서 공통적으로 <그림>, <표>가 등장하므로 분석상 의미없는 단어를 삭제했습니다.
origin_df = origin_df.applymap(lambda x: str(x).replace('<그림>', '') if pd.notnull(x) else x)
origin_df = origin_df.applymap(lambda x: str(x).replace('<표>', '') if pd.notnull(x) else x)


- 16대에서 위원을 표현할 때 띄어쓰기가 제대로 되어있지 않아 추후 과정에서 문제가 생길 수 있습니다. 
- 앞뒤로 공백을 추가한 뒤, 공백이 두 번 중첩된 경우 이를 하나로 줄였습니다.

### 위와같이 진행할 경우, 위원장에서 문제가 생길 수 있기 때문에 위원장은 이 코드는 현 상태에서 사용하지 않는 것으로 합니다.
origin_df = origin_df.applymap(lambda x: str(x).replace('위원', ' 위원 ') if pd.notnull(x) else x)
origin_df = origin_df.applymap(lambda x: str(x).replace('  ', ' ') if pd.notnull(x) else x)

In [93]:
origin_df.iloc[96,0]

'국회회의록_16대_213회_1차_법제사법위원회.csv 0(제213회－법제사법제1차)  (제213회－법제사법제1차)(10시12분 개의)◯위원장 박헌기 성원이 되었으므로 제213회국회 임시회 제1차 법제사법위원회를 개의하겠습니다.  먼저 입법조사관의 보고사항이 있겠습니다. ◯입법조사관 문제풍 보고를 드리겠습니다. (보고사항은 끝에 실음)◯위원장 박헌기 의사진행에 들어가기 전에 새로 우리 위원회에 보임되신 새천년민주당 소속 이희규 위원님을 환영하면서 인사말씀을 간단히 듣도록 하겠습니다. 인사해 주시기 바랍니다. ◯이희규위원 반갑습니다. 새천년민주당의 이희규입니다.  잘 모릅니다. 그저 와서 여러 위원님들 하시는 것을 보고 배우고 소임을 다하도록 하겠습니다. 잘 부탁합니다.  1. 소위원회구성의건(10시13분)◯위원장 박헌기 의사일정 제1항 소위원회구성의건을 상정하겠습니다. " 여러 위원님들께서 잘 아시는 바와 같이 국회법에 각 위원회는 상설소위원회를 구성하여 안건심사를 하도록 되어 있습니다마는 우리 국회 위원회 전체적으로 상설소위원회 구성에 관한 구체적인 계획안이 아직 마련되지 못한 상태에 있습니다. 따라서 부득이 종전에 구성하여 운영하여 온 대로 법안심사제1소위원회는 법사위원회 소관 고유법안을 다루도록 하고  법안심사제2소위원회는 타 상임위원회 소관법안의 체계‧자구심사를 맡도록 하고  예산안심사소위원회에서는 예산을  그리고 청원심사소위원회는 청원을 심사하도록 간사간에 합의가 되었습니다. "" 그리고 각 안건심사소위원 구성에 관해서는 간사간에 협의하여 미리 각 위원님들께 배부해 드린 바가 있습니다. 여러 위원님들께서 양해하여 주신다면 간사간에 합의된 대로 법안심사제1‧제2소위원회  예산안심사소위원회  청원심사소위원회의 소위원장과 위원 선임을 배부해 드린 유인물과 같이 구성하고자 하는데 이의 없으십니까? " (｢없습니다｣하는 위원 있음) 이의 없으시면 가결되었음을 선포합니다.  소위원회 구성 명단은 속기록에 게재하도록 하겠습니다. (명단은 끝에 실음) 2. 약사법중

## origin_df 파일 저장

In [94]:
file_path = 'D:\MJ 데이터 공모전 로우 데이터\\origins\\17_origin_df.csv'
origin_df.to_csv(file_path, index=False)

## orgin_df 불러오기

In [3]:
# CSV 파일 경로
file_path = 'D:\\MJ 데이터 공모전 로우 데이터\\origins\\17_origin_df.csv'

# CSV 파일 불러오기
origin_df = pd.read_csv(file_path, encoding='utf-8', header=0)

In [95]:
origin_df

,0
0,국회회의록_16대_212회_1차_건설교통위원회.csv 0(제212회―건설교통제1차)...
1,국회회의록_16대_212회_1차_과학기술정보통신위원회.csv 0(제212회－과학기술...
2,국회회의록_16대_212회_1차_교육위원회.csv 0(제212회－교육제1차) (제...
3,국회회의록_16대_212회_1차_국무총리(이한동)임명동의에관한인사청문특별위원회.cs...
4,국회회의록_16대_212회_1차_국방위원회.csv 0(제212회-국방제1차) (...
...,...
2097,국회회의록_16대_246회_1차_국방위원회.csv 0제246회-국방제1차 제246...
2098,국회회의록_16대_246회_1차_국회본회의.csv 0제246회－제1차 제246회－...
2099,국회회의록_16대_246회_1차_통일외교통상위원회.csv 0제246회－통일외교통상제...
2100,국회회의록_16대_246회_2차_국회본회의.csv 0제246회－제2차 제246회－...


## 과정 설명

1. origin_df를 불러온다.
    - origin_df가 망가지면 다시 불러와야 하므로 copied_df를 사용한다.
2. csv 파일 전체가 하나의 셀에 입력되어 있으므로 
    - 파일의 이름을 떼어내서 행의 카테고리로 사용하기 위해 간단한 전처리 작업을 거친다.
        - '회_'를 기준으로 스플릿하여 '차_'로 스플릿 할 때의 불상사를 막는다 ex) '자동차_', '소방차_'...
        - '파일의 제목을 헷갈리게 만드는 '국정조사', '국정감사', '개회식'은 삭제한다.
        - '(1)', '(2)' 등 문서의 개수를 나타내는 정보를 삭제한다.
        - 카테고리 뒤에 붙어있는 의미없는 공백, 탭 등의 개행문자를 rstrip()으로 삭제한다.
        - copied_df를 정렬하고 카테고리에 중복이 없는지 확인한다.
        - 확인이 완료되면 속기록 내부에서 발언자를 구분하는 '◯'를 사용하여 스플릿하고 grouped_df에 할당한다.

### origin_df 불러오기

In [96]:
copied_df = origin_df.copy()

In [97]:
copied_df

,0
0,국회회의록_16대_212회_1차_건설교통위원회.csv 0(제212회―건설교통제1차)...
1,국회회의록_16대_212회_1차_과학기술정보통신위원회.csv 0(제212회－과학기술...
2,국회회의록_16대_212회_1차_교육위원회.csv 0(제212회－교육제1차) (제...
3,국회회의록_16대_212회_1차_국무총리(이한동)임명동의에관한인사청문특별위원회.cs...
4,국회회의록_16대_212회_1차_국방위원회.csv 0(제212회-국방제1차) (...
...,...
2097,국회회의록_16대_246회_1차_국방위원회.csv 0제246회-국방제1차 제246...
2098,국회회의록_16대_246회_1차_국회본회의.csv 0제246회－제1차 제246회－...
2099,국회회의록_16대_246회_1차_통일외교통상위원회.csv 0제246회－통일외교통상제...
2100,국회회의록_16대_246회_2차_국회본회의.csv 0제246회－제2차 제246회－...


### 카테고리 생성을 위한 전처리

In [98]:
copied_df = copied_df[0].str.replace('국정조사_', '').str.replace('국정감사_', '').str.replace('개회식_', '')

copied_df = pd.DataFrame(copied_df)

copied_df = copied_df[0].str.split('회_', expand=True)

copied_df = copied_df.drop([0, 1], axis=1)

copied_df = copied_df[2].str.split('.csv', expand=True)

copied_df[0] = copied_df[0].apply(lambda x: re.sub(r'\d{1,2}차_', '', x))

copied_df = pd.DataFrame(copied_df)

# 정규식 패턴과 숫자를 삭제하는 함수 정의
def remove_numbers_in_parentheses(text):
    import re
    pattern = r"\(\d+\)"
    return re.sub(pattern, '', str(text))

copied_df[0] = copied_df[0].apply(remove_numbers_in_parentheses)

copied_df = pd.DataFrame(copied_df)

copied_df[0] = copied_df[0].str.rstrip()

copied_df = copied_df.sort_values(by=copied_df.columns[0])

### 나눠진 카테고리의 중복이 없는지 확인

In [99]:
from collections import OrderedDict

data_list = copied_df.iloc[:, 0].tolist()
unique_data_list = list(OrderedDict.fromkeys(data_list).keys())

In [60]:
unique_data_list

['2002년월드컵등국제경기대회지원특별위원회',
 '2003년대구하계유니버시아드대회지원특별위원회',
 '2010년세계박람회유치특별위원회',
 '2010년평창동계올림픽유치특별위원회',
 '감사원장(윤성식)임명동의에관한인사청문특별위원회',
 '감사원장(전윤철)임명동의에관한인사청문특별위원회',
 '건설교통위원회',
 '과거사진상규명에관한특별위원회',
 '과학기술정보통신위원회',
 '교육위원회',
 '국군부대의이라크전쟁파견동의안에관한전원위원회',
 '국무총리(김석수)임명동의에관한인사청문특별위원회',
 '국무총리(이한동)임명동의에관한인사청문특별위원회',
 '국무총리(장대환)임명동의에관한인사청문특별위원회',
 '국무총리(장상)임명동의에관한인사청문특별위원회',
 '국무총리후보자(고건)에관한인사청문특별위원회',
 '국방위원회',
 '국회본회의',
 '국회상임위원회위원정수에관한규칙개정특별위원회',
 '국회운영위원회',
 '기후변화협약대책특별위원회',
 '남북관계발전지원특별위원회',
 '남북정상회담관련결의안기초특별위원회',
 '농림해양수산위원회',
 '대법관(고현철)임명동의에관한인사청문특별위원회',
 '대법관(김용담)임명동의에관한인사청문특별위원회',
 '문화관광위원회',
 '미래전략특별위원회',
 '법제사법위원회',
 '보건복지위원회',
 '보훈특별위원회',
 '산업자원위원회',
 '실업대책특별위원회',
 '여성위원회',
 '여성특별위원회',
 '예산결산특별위원회',
 '윤리특별위원회',
 '인사청문특별위원회',
 '인사청문특별위원회(헌재)',
 '일본역사교과서왜곡시정을위한특별위원회',
 '장애인특별위원회',
 '재정경제위원회',
 '재정제도개혁특별위원회',
 '재해대책특별위원회',
 '정무위원회',
 '정보위원회',
 '정치개혁특별위원회',
 '중앙선거관리위원회위원(김영철)선출에관한인사청문특별위원회',
 '중앙선거관리위원회위원선출에관한인사청문특별위원회',
 '통일외교통상위원회',
 '행정자치위원회',
 '헌법재판소재판관(이상경)선출에관한인사청문특별위원회',
 '환경노동위원회'

In [100]:
copied_df

,0,1
518,2002년월드컵등국제경기대회지원특별위원회,(제220회－2002년월드컵등국제경기대회지원특제2차) (제220회－2002년월드컵...
851,2002년월드컵등국제경기대회지원특별위원회,(제225회-2002년월드컵특제4차)(제225회-2002년월드컵특제4차) (10...
400,2002년월드컵등국제경기대회지원특별위원회,(제218회-2002년월드컵등국제경기대회지원특제1차)(제218회-2002년월드컵등국...
617,2002년월드컵등국제경기대회지원특별위원회,(제222회-2002년월드컵특제1차)(제222회-2002년월드컵특제1차) (10...
1474,2003년대구하계유니버시아드대회지원특별위원회,제238회-2003년대구하계특제1차 제238회-2003년대구하계특제1차(10시19...
...,...,...
489,환경노동위원회,(제219회－환경로동제1차) (제219회－환경로동제1차) (14시30분 개의)◯위...
1431,환경노동위원회,제236회-환경노동제2차 제236회-환경노동제2차(10시14분 개의)◯위원장 송훈...
1017,환경노동위원회,(제227회－환경로동제2차) (제227회－환경로동제2차) (10시16분 개의)◯위...
1754,환경노동위원회,제243회-환경노동제12차 제243회-환경노동제12차(10시51분 개의)◯위원장 ...


In [101]:
grouped_df = copied_df.groupby(copied_df.columns[0])[copied_df.columns[1]].apply(list).reset_index()

In [102]:
grouped_df

,0,1
0,2002년월드컵등국제경기대회지원특별위원회,[(제220회－2002년월드컵등국제경기대회지원특제2차) (제220회－2002년월드...
1,2003년대구하계유니버시아드대회지원특별위원회,[제238회-2003년대구하계특제1차 제238회-2003년대구하계특제1차(10시1...
2,2010년세계박람회유치특별위원회,[(제220회－2010년세계박람회유치특제1차) (제220회－2010년세계박람회유치...
3,2010년평창동계올림픽유치특별위원회,[제238회-2010년평창동계올림픽특위제2차 제238회-2010년평창동계올림픽특위...
4,감사원장(윤성식)임명동의에관한인사청문특별위원회,[제243회－인사청문특제1차 제243회－인사청문특제1차 (보고)(09시15분)◯입...
5,감사원장(전윤철)임명동의에관한인사청문특별위원회,[제243회－인사청문특제2차 제243회－인사청문특제2차(10시01분 개의)◯위원장...
6,건설교통위원회,[제240회－건설교통제2차 제240회－건설교통제2차(14시05분 개의)◯위원장 신...
7,과거사진상규명에관한특별위원회,[제243회-과거사진상규명특제5차 제243회-과거사진상규명특제5차(10시30분 개...
8,과학기술정보통신위원회,[(제215회－과학기술정보통신제5차) (제215회－과학기술정보통신제5차) (00시...
9,교육위원회,[제239회-교육제1차 제239회-교육제1차(14시11분 개의)◯위원장 윤영탁 성...


In [103]:
grouped_df.iloc[2,1]

['(제220회－2010년세계박람회유치특제1차)  (제220회－2010년세계박람회유치특제1차) (보 고)(09시57분)◯수석전문위원 금기영 수석전문위원입니다. 국회법 제47조제2항의 규정에 의하면 특별위원회의 위원장이 선임될 때까지는 위원 중 연장자가 위원장의 직무를 대행하도록 규정하고 있습니다. 따라서 참석위원님 중 연장위원이신 민봉기 위원님 나오셔서 위원장 선출을 위한 회의를 주재하여 주시기 바랍니다.(09시58분 개의)◯위원장직무대행 민봉기 성원이 되었으므로 220회국회 임시회 제1차 2010년세계박람회유치특별위원회를 개의하겠습니다. 본 위원이 2010년에 개최 예정인 세계박람회를 우리나라에 유치하기 위하여 구성된 2010년세계박람회유치특별위원회의 위원장을 선임하기 위한 회의를 주재하게 된 데 대해서 매우 영광스럽게 생각합니다. 회의진행 과정에서 다소 미숙한 점이 있더라도 여러 위원님들께서 널리 양해하여 주시고 적극적으로 협조하여 주시기 바랍니다. 그러면 먼저 입법조사관으로부터 보고가 있겠습니다.◯입법조사관 조이제 보고사항을 말씀드리겠습니다.(보고사항은 끝에 실음) 1. 특별위원장선임의건(10시00분)◯위원장직무대행 민봉기 그러면 의사일정 제1항 특별위원장선임의건을 상정합니다. 여러 위원님께서 잘 아시는 바와 같이 특별위원회의 위원장 선임은 국회법 제47조제1항의 규정에 의하여 위원회에서 호선하고 본회의에 보고하도록 되어 있습니다.  지금까지의 관례상 특별위원회의 위원장은 구두호천에 의해 선임되어 온 것으로 알고 있습니다. 여러 위원님께서 선임방법에 관하여 의견이 있으시면 말씀해 주시기 바랍니다.◯이주영위원 이주영 위원입니다. 방금 임시 위원장님께서 위원장 선임방법의 관례가 구두호천이라고 말씀해 주셨습니다. 이 관례를 존중하는 뜻에서 위원장 선임은 구두호천으로 선임할 것을 동의합니다.◯위원장직무대행 민봉기 방금 이주영 위원님으로부터 위원장 선임방법은 구두호천으로 하자는 동의가 있었습니다. 이 동의에 대하여 재청이 있으십니까? (｢재청입니다｣ 하는 위원 있

In [104]:
grouped_df[1] = grouped_df[1].apply(lambda lst: [item for sublist in lst for item in sublist.split('◯')] if isinstance(lst, list) else lst)

### 조사대상 국회의원 1899명의 이름

In [105]:
names = ["이명박",	"강경식",	"강삼재",	"강성재",	"강용식",	"강재섭",	"강종희",	"강창희",	"강현욱",	"구천서",	"국창근",	"권기술",	"권노갑",	"권수창",	"권영자",	"권오을",	"권익현",	"권정달",	"권철현",	"길승흠",	"김경재 ",	"김고성",	"김광수",	"김광원",	"김근태",	"김기수",	"김기재",	"김기춘",	"김길환",	"김덕",	"김덕룡",	"김도언",	"김동욱",	"김동주",	"김명규",	"김명섭",	"김명윤",	"김무성",	"김문수",	"김민석",	"김범명",	"김병태",	"김복동",	"김봉호",	"김상우",	"김상현",	"김석원",	"김선길",	"김성곤",	"김수한",	"김영구",	"김영배",	"김영선",	"김영일",	"김영준",	"김영진",	"김영진",	"김영환",	"김옥두",	"김용갑",	"김용환",	"김운환",	"김원길",	"김윤환",	"김의재",	"김인곤",	"김인영",	"김일윤",	"김일주",	"김재천",	"김정수",	"김정숙",	"김종배",	"김종필",	"김종하",	"김종학",	"김종호",	"김중위",	"김진배",	"김진재",	"김찬우",	"김찬진",	"김철",	"김충일",	"김충조",	"김칠환",	"김태랑",	"김태식",	"김태호",	"김학원",	"김한길",	"김허남",	"김현욱",	"김형오",	"김호일",	"김홍신",	"김홍일",	"나오연",	"남경필",	"남궁진",	"남평우",	"노기태",	"노무현",	"노승우",	"류선호",	"류종수",	"맹형규",	"목요상",	"박관용",	"박광태",	"박구일",	"박근혜",	"박명환",	"박범진",	"박상규",	"박상복",	"박상천",	"박성범",	"박세직",	"박세환",	"박승국",	"박시균",	"박신원",	"박우병",	"박원홍",	"박정수",	"박정훈",	"박종근",	"박종우",	"박종웅",	"박주천",	"박준규",	"박찬주",	"박창달",	"박철언",	"박태준",	"박헌기",	"박희태",	"방용석",	"배종무",	"백남치",	"백승홍",	"변웅전",	"변정일",	"서상목",	"서석재",	"서정화",	"서정화",	"서청원",	"서한샘",	"서훈",	"설훈",	"손세일",	"손학규",	"송업교",	"송현섭",	"송훈석",	"신경식",	"신기남",	"신기하",	"신낙균",	"신상우",	"신영국",	"신영균",	"심정구",	"안동선",	"안상수",	"안상수",	"안재홍",	"안택수",	"양성철",	"양정규",	"어준선",	"오세응",	"오양순",	"오용운",	"오장섭",	"원유철",	"유용태",	"유재건",	"유흥수",	"윤원중",	"윤철상",	"윤한도",	"이강두",	"이강희",	"이건개",	"이경재",	"이국헌",	"이규정",	"이규택",	"이긍규",	"이길재",	"이동복",	"이동원",	"이만섭",	"이미경",	"이병희",	"이부영",	"이사철",	"이상득",	"이상만",	"이상배",	"이상수",	"이상현",	"이상희",	"이석현",	"이성재",	"이성호",	"이세기",	"이수인",	"이신범",	"이양희",	"이영일",	"이완구",	"이용삼",	"이우재",	"이웅희",	"이원범",	"이원복",	"이윤성",	"이윤수",	"이응선",	"이의익",	"이인구",	"이재명",	"이재선",	"이재오",	"이재창",	"이정무",	"이중재",	"이찬진",	"이태섭",	"이택석",	"이한동",	"이해구",	"이해봉",	"이해찬",	"이협",	"이형배",	"이홍구",	"이회창",	"이훈평",	"임복진",	"임인배",	"임진출",	"임채정",	"장성원",	"장영달",	"장영철",	"장을병",	"장재식",	"전석홍",	"전용원",	"정균환",	"정동영",	"정동채",	"정몽준",	"정문화",	"정상구",	"정상천",	"정석모",	"정세균",	"정영훈",	"정우택",	"정의화",	"정일영",	"정재문",	"정재철",	"정창화",	"정한용",	"정형근",	"정호선",	"정희경",	"제정구",	"조성준",	"조세형",	"조순",	"조순승",	"조순형",	"조영재",	"조웅규",	"조익현",	"조중연",	"조진형",	"조찬형",	"조철구",	"조한천",	"조홍규",	"주양자",	"주진우",	"지대섭",	"차수명",	"채영석",	"천용택",	"천정배",	"최병렬",	"최선영",	"최연희",	"최재승",	"최형우",	"최희준",	"추미애",	"하경근",	"하순봉",	"한광옥",	"한승수",	"한영수",	"한영애",	"한이헌",	"한호선",	"한화갑",	"함석재",	"함종한",	"허남훈",	"허대범",	"허화평",	"현경대",	"홍문종",	"홍사덕",	"홍인길",	"황규선",	"황낙주",	"황병태",	"황성균",	"황승민",	"황우여",	"황학수",	"강봉균",	"강삼재",	"강성구",	"강숙자",	"강신성일",	"강운태",	"강인섭",	"강재섭",	"강창성",	"강창희",	"강현욱",	"고진부",	"고흥길",	"구종태",	"권기술",	"권영세",	"권오을",	"권철현",	"권태망",	"김경재 ",	"김경천",	"김광원",	"김근태",	"김기배",	"김기재",	"김기춘",	"김덕규",	"김덕룡",	"김덕배",	"김동욱",	"김락기",	"김만제",	"김명섭",	"김무성",	"김문수",	"김민석",	"김방림",	"김병호",	"김부겸",	"김상현",	"김성순",	"김성조",	"김성호",	"김영구",	"김영구",	"김영선",	"김영일",	"김영진",	"김영춘",	"김영환",	"김옥두",	"김용갑",	"김용균",	"김용학",	"김용환",	"김운용",	"김원기",	"김원길",	"김원웅",	"김일윤",	"김정부",	"김정숙",	"김종필",	"김종하",	"김종호",	"김진재",	"김찬우",	"김충조",	"김태식",	"김태호",	"김택기",	"김학송",	"김학원",	"김한길",	"김형오",	"김홍신",	"김홍일",	"김화중",	"김황식",	"김효석",	"김희선",	"나오연",	"남경필",	"남궁석",	"도종이",	"류시민",	"맹형규",	"목요상",	"문석호",	"문희상",	"민봉기",	"박경섭",	"박관용",	"박광태",	"박근혜",	"박금자",	"박명환",	"박병석",	"박병윤",	"박상규",	"박상천",	"박상희",	"박세환",	"박승국",	"박시균",	"박양수",	"박원홍",	"박인상",	"박재욱",	"박종근",	"박종완",	"박종우",	"박종웅",	"박종희",	"박주선",	"박주천",	"박진",	"박창달",	"박헌기",	"박혁규",	"박희태",	"배기선",	"배기운",	"백승홍",	"변웅전",	"서병수",	"서상섭",	"서영훈",	"서정화",	"서청원",	"설송웅",	"설훈",	"손태인",	"손학규",	"손희정",	"송광호",	"송병대",	"송석찬",	"송영길",	"송영진",	"송훈석",	"신경식",	"신계륜",	"신기남",	"신영국",	"신영균",	"신현태",	"심규섭",	"심규철",	"심재권",	"심재철",	"안경률",	"안대륜",	"안동선",	"안상수",	"안상현",	"안영근",	"안택수",	"안희옥",	"양승부",	"양정규",	"엄호성",	"오경훈",	"오세훈",	"오영식",	"오장섭",	"원유철",	"원철희",	"원희룡",	"유삼남",	"유용태",	"유재건",	"유재규",	"유한열",	"유흥수",	"윤경식",	"윤두환",	"윤여준",	"윤영탁",	"윤철상",	"윤한도",	"이강두",	"이강래",	"이경재",	"이규택",	"이근진",	"이낙연",	"이만섭",	"이만재",	"이미경",	"이방호",	"이병석",	"이부영",	"이상득",	"이상배",	"이상수",	"이상희",	"이성헌",	"이승철",	"이양희",	"이연숙",	"이완구",	"이용삼",	"이우재",	"이원성",	"이원창",	"이원형",	"이윤성",	"이윤수",	"이인기",	"이인제",	"이재선",	"이재오",	"이재정",	"이재창",	"이정일",	"이종걸",	"이종성",	"이주영",	"이창복",	"이한구",	"이한동",	"이해구",	"이해봉",	"이해찬",	"이협",	"이호웅",	"이회창",	"이훈평",	"이희규",	"임인배",	"임종석",	"임진출",	"임채정",	"임태희",	"장광근",	"장성원",	"장영달",	"장재식",	"장태완",	"전갑길",	"전용원",	"전용학",	"전재희",	"정갑윤",	"정균환",	"정대철",	"정동영",	"정동채",	"정몽준",	"정문화",	"정범구",	"정병국",	"정세균",	"정우택",	"정의화",	"정장선",	"정진석",	"정창화",	"정철기",	"정형근",	"조배숙",	"조부영",	"조성준",	"조순형",	"조웅규",	"조재환",	"조정무",	"조한천",	"조희욱",	"주진우",	"천용택",	"천정배",	"최돈웅",	"최명헌",	"최병국",	"최병렬",	"최선영",	"최연희",	"최영희",	"최용규",	"최재승",	"추미애",	"하순봉",	"한명숙",	"한승수",	"한충수",	"한화갑",	"함석재",	"함승희",	"허운나",	"허태열",	"현경대",	"현승일",	"홍문종",	"홍사덕",	"홍재형",	"홍준표",	"황승민",	"황우여",	"황창주",	"강기갑",	"강기정",	"강길부",	"강봉균",	"강성종",	"강재섭",	"강창일",	"강혜숙",	"고경화",	"고조흥",	"고진화",	"고흥길",	"고희선",	"공성진",	"곽성문",	"구논회",	"권경석",	"권선택",	"권영길",	"권영세",	"권오을",	"권철현",	"김광원",	"김교흥",	"김근태",	"김기춘",	"김기현",	"김낙성",	"김낙순",	"김덕규",	"김덕룡",	"김동철",	"김명자",	"김명주",	"김무성",	"김문수",	"김병호",	"김부겸",	"김석준",	"김선미",	"김성곤",	"김성조",	"김송자",	"김애실",	"김양수",	"김영대",	"김영덕",	"김영선",	"김영숙",	"김영주",	"김영춘",	"김용갑",	"김우남",	"김원기",	"김원웅",	"김재경",	"김재원",	"김재윤",	"김재홍",	"김정권",	"김정부",	"김정훈",	"김종률",	"김종인",	"김진표",	"김춘진",	"김충환",	"김태년",	"김태홍",	"김태환",	"김학송",	"김학원",	"김한길",	"김혁규",	"김현미",	"김형오",	"김형주",	"김홍업",	"김홍일",	"김효석",	"김희선",	"김희정",	"나경원",	"남경필",	"노영민",	"노웅래",	"노현송",	"노회찬",	"단병호",	"류근찬",	"류선호",	"류시민",	"맹형규",	"문병호",	"문석호",	"문학진",	"문희",	"문희상",	"민병두",	"박계동",	"박근혜",	"박기춘",	"박명광",	"박병석",	"박상돈",	"박성범",	"박세일",	"박세환",	"박순자",	"박승환",	"박영선",	"박재완",	"박종근",	"박진",	"박찬석",	"박찬숙",	"박형준",	"박홍수",	"박희태",	"배기선",	"배일도",	"백원우",	"변재일",	"서갑원",	"서병수",	"서상기",	"서재관",	"서혜석",	"선병렬",	"손봉숙",	"송영길",	"송영선",	"신계륜",	"신국환",	"신기남",	"신명",	"신상진",	"신중식",	"신학용",	"심대평",	"심상정",	"심재덕",	"심재엽",	"심재철",	"안경률",	"안명옥",	"안민석",	"안병엽",	"안상수",	"안영근",	"안택수",	"안홍준",	"양승조",	"양형일",	"엄호성",	"염동연",	"오영식",	"오제세",	"우상호",	"우원식",	"우윤근",	"우제창",	"우제항",	"원혜영",	"원희룡",	"유기준",	"유기홍",	"유승민",	"유승희",	"유인태",	"유재건",	"유정복",	"유필우",	"윤건영",	"윤원호",	"윤호중",	"이강두",	"이강래",	"이경숙",	"이경재",	"이계경",	"이계안",	"이계진",	"이광재",	"이광철",	"이군현",	"이규택",	"이근식",	"이기우",	"이낙연",	"이명규",	"이목희",	"이미경",	"이방호",	"이병석",	"이상경",	"이상득",	"이상민",	"이상배",	"이상열",	"이석현",	"이성구",	"이성권",	"이승희",	"이시종",	"이영순",	"이영호",	"이용희",	"이원복",	"이원영",	"이윤성",	"이은영",	"이인기",	"이인영",	"이인제",	"이재오",	"이재웅",	"이재창",	"이정일",	"이종걸",	"이종구",	"이주영",	"이주호",	"이진구",	"이한구",	"이해봉",	"이해찬",	"이혜훈",	"이호웅",	"이화영",	"임인배",	"임종석",	"임종인",	"임채정",	"임태희",	"임해규",	"장경수",	"장복심",	"장영달",	"장윤석",	"장향숙",	"전병헌",	"전여옥",	"전재희",	"정갑윤",	"정덕구",	"정동채",	"정두언",	"정몽준",	"정문헌",	"정병국",	"정봉주",	"정성호",	"정세균",	"정의용",	"정의화",	"정장선",	"정종복",	"정진석",	"정진섭",	"정청래",	"정형근",	"정화원",	"정희수",	"제종길",	"조경태",	"조배숙",	"조성래",	"조성태",	"조순형",	"조일현",	"조정식",	"주성영",	"주승용",	"주호영",	"지병문",	"진수희",	"진영",	"차명진",	"채수찬",	"채일병",	"천영세",	"천정배",	"최경환",	"최구식",	"최규성",	"최규식",	"최병국",	"최성",	"최순영",	"최연희",	"최용규",	"최인기",	"최재성",	"최재천",	"최철국",	"한광원",	"한명숙",	"한병도",	"한선교",	"한화갑",	"허천",	"허태열",	"현경병",	"현애자",	"홍문표",	"홍미영",	"홍재형",	"홍준표",	"홍창선",	"황우여",	"황진하",	"강기갑",	"강기정",	"강길부",	"강명순",	"강봉균",	"강석호",	"강성종",	"강성천",	"강승규",	"강용석",	"강운태",	"강창일",	"고승덕",	"고흥길",	"공성진",	"곽정숙",	"구상찬",	"권경석",	"권선택",	"권성동",	"권영길",	"권영세",	"권영진",	"권택기",	"김광림",	"김금래",	"김기현",	"김낙성",	"김동성",	"김동철",	"김무성",	"김부겸",	"김상희",	"김선동",	"김선동",	"김성곤",	"김성동",	"김성수",	"김성순",	"김성식",	"김성조",	"김성태",	"김성회",	"김세연",	"김소남",	"김영록",	"김영선",	"김영우",	"김영진",	"김영환",	"김옥이",	"김용구",	"김용태",	"김우남",	"김유정",	"김을동",	"김일윤",	"김장수",	"김재경",	"김재균",	"김재윤",	"김정",	"김정권",	"김정훈",	"김종률",	"김진애",	"김진표",	"김창수",	"김춘진",	"김충조",	"김충환",	"김태원",	"김태호",	"김태환",	"김학송",	"김학용",	"김학재",	"김형오",	"김혜성",	"김호연",	"김효석",	"김효재",	"김희철",	"나경원",	"나성린",	"남경필",	"노영민",	"노철래",	"류근찬",	"류선호",	"문학진",	"문희상",	"박근혜",	"박기춘",	"박대해",	"박민식",	"박병석",	"박보환",	"박상돈",	"박상은",	"박상천",	"박선숙",	"박선영",	"박순자",	"박영선",	"박영아",	"박우순",	"박은수",	"박종근",	"박주선",	"박준선",	"박지원",	"박진",	"박희태",	"배영식",	"배은희",	"백성운",	"백원우",	"백재현",	"변웅전",	"변재일",	"서갑원",	"서병수",	"서상기",	"서종표",	"선경식",	"성윤환",	"손범규",	"손숙미",	"손학규",	"송광호",	"송민순",	"송영길",	"송영선",	"송훈석",	"신건",	"신낙균",	"신상진",	"신성범",	"신영수",	"신지호",	"신학용",	"심대평",	"심재철",	"안경률",	"안규백",	"안민석",	"안상수",	"안형환",	"안홍준",	"안효대",	"양승조",	"여상규",	"오제세",	"우윤근",	"우제창",	"원유철",	"원혜영",	"원희룡",	"원희목",	"유기준",	"유성엽",	"유승민",	"유원일",	"유일호",	"유재중",	"유정복",	"유정현",	"윤상일",	"윤상현",	"윤석용",	"윤영",	"윤진식",	"이강래",	"이경재",	"이계진",	"이광재",	"이군현",	"이낙연",	"이달곤",	"이두아",	"이명규",	"이명수",	"이미경",	"이범관",	"이범래",	"이병석",	"이사철",	"이상권",	"이상득",	"이상민",	"이석현",	"이성남",	"이성헌",	"이시종",	"이애주",	"이영애",	"이영애",	"이용경",	"이용섭",	"이용희",	"이윤석",	"이윤성",	"이은재",	"이인기",	"이인제",	"이재선",	"이재오",	"이정선",	"이정현",	"이정희",	"이종걸",	"이종구",	"이종혁",	"이주영",	"이진복",	"이진삼",	"이찬열",	"이철우",	"이춘석",	"이춘식",	"이학재",	"이한구",	"이한성",	"이해봉",	"이혜훈",	"이화수",	"이회창",	"임동규",	"임두성",	"임영호",	"임태희",	"임해규",	"장광근",	"장병완",	"장세환",	"장윤석",	"장제원",	"전병헌",	"전여옥",	"전재희",	"전현희",	"전혜숙",	"정갑윤",	"정동영",	"정두언",	"정몽준",	"정미경",	"정범구",	"정병국",	"정세균",	"정수성",	"정양석",	"정영희",	"정옥임",	"정의화",	"정장선",	"정진석",	"정진섭",	"정태근",	"정하균",	"정해걸",	"정희수",	"조경태",	"조문환",	"조배숙",	"조순형",	"조승수",	"조영택",	"조원진",	"조윤선",	"조전혁",	"조정식",	"조진래",	"조진형",	"조해진",	"주광덕",	"주성영",	"주승용",	"주호영",	"진성호",	"진수희",	"진영",	"차명진",	"천정배",	"최경환",	"최경희",	"최구식",	"최규성",	"최규식",	"최문순",	"최병국",	"최연희",	"최영희",	"최인기",	"최재성",	"최종원",	"최철국",	"추미애",	"한기호",	"한선교",	"허원제",	"허천",	"허태열",	"현경병",	"현기환",	"홍사덕",	"홍영표",	"홍일표",	"홍재형",	"홍정욱",	"홍준표",	"홍희덕",	"황영철",	"황우여",	"황진하",	"강기윤",	"강기정",	"강길부",	"강동원",	"강석호",	"강석훈",	"강은희",	"강창일",	"강창희",	"경대수",	"고희선",	"권성동",	"권은희",	"권은희",	"길정우",	"김경협",	"김관영",	"김광림",	"김광진",	"김기선",	"김기식",	"김기준",	"김기현",	"김도읍",	"김동완",	"김동철",	"김명연",	"김무성",	"김미희",	"김민기",	"김상민",	"김상훈",	"김상희",	"김선동",	"김성곤",	"김성주",	"김성찬",	"김성태",	"김세연",	"김승남",	"김영록",	"김영우",	"김영주",	"김영환",	"김용남",	"김용익",	"김용태",	"김우남",	"김윤덕",	"김을동",	"김장실",	"김재경",	"김재연",	"김재원",	"김재윤",	"김정록",	"김정훈",	"김제남",	"김제식",	"김종태",	"김종훈",	"김진태",	"김진표",	"김춘진",	"김태년",	"김태원",	"김태호",	"김태환",	"김태흠",	"김학용",	"김한길",	"김한표",	"김현",	"김현미",	"김현숙",	"김회선",	"김희국",	"김희정",	"나경원",	"나성린",	"남경필",	"남인순",	"노영민",	"노웅래",	"노철래",	"노회찬",	"도종환",	"류성걸",	"류지영",	"문대성",	"문병호",	"문재인",	"문정림",	"문희상",	"민병두",	"민병주",	"민현주",	"민홍철",	"박광온",	"박근혜",	"박기춘",	"박남춘",	"박대동",	"박대출",	"박덕흠",	"박맹우",	"박명재",	"박민수",	"박민식",	"박범계",	"박병석",	"박상은",	"박성호",	"박성효",	"박수현",	"박영선",	"박완주",	"박원석",	"박윤옥",	"박인숙",	"박주선",	"박지원",	"박창식",	"박혜자",	"박홍근",	"배기운",	"배덕광",	"배재정",	"백군기",	"백재현",	"변재일",	"부좌현",	"서기호",	"서병수",	"서상기",	"서영교",	"서용교",	"서청원",	"설훈",	"손인춘",	"송광호",	"송영근",	"송호창",	"신경림",	"신경민",	"신계륜",	"신기남",	"신동우",	"신문식",	"신상진",	"신성범",	"신의진",	"신정훈",	"신학용",	"심상정",	"심윤조",	"심재권",	"심재철",	"심학봉",	"안규백",	"안민석",	"안상수",	"안종범",	"안철수",	"안홍준",	"안효대",	"양승조",	"양창영",	"여상규",	"염동열",	"오병윤",	"오신환",	"오영식",	"오제세",	"우상호",	"우원식",	"우윤근",	"원유철",	"원혜영",	"유기준",	"유기홍",	"유대운",	"유성엽",	"유승민",	"유승우",	"유승희",	"유은혜",	"유의동",	"유인태",	"유일호",	"유재중",	"유정복",	"윤관석",	"윤금순",	"윤명희",	"윤상현",	"윤영석",	"윤재옥",	"윤진식",	"윤호중",	"윤후덕",	"은수미",	"이강후",	"이개호",	"이군현",	"이낙연",	"이노근",	"이만우",	"이명수",	"이목희",	"이미경",	"이병석",	"이상규",	"이상민",	"이상일",	"이상직",	"이석기",	"이석현",	"이언주",	"이에리사",	"이완구",	"이완영",	"이용섭",	"이우현",	"이운룡",	"이원욱",	"이윤석",	"이이재",	"이인영",	"이인제",	"이자스민",	"이장우",	"이재영",	"이재오",	"이정현",	"이종걸",	"이종배",	"이종진",	"이종훈",	"이주영",	"이진복",	"이찬열",	"이채익",	"이철우",	"이춘석",	"이학영",	"이학재",	"이한구",	"이한성",	"이해찬",	"이헌승",	"이현재",	"인재근",	"임내현",	"임수경",	"장병완",	"장윤석",	"장정은",	"장하나",	"전병헌",	"전순옥",	"전정희",	"전하진",	"전해철",	"정갑윤",	"정두언",	"정몽준",	"정문헌",	"정미경",	"정병국",	"정성호",	"정세균",	"정수성",	"정용기",	"정우택",	"정윤숙",	"정의화",	"정진후",	"정청래",	"정호준",	"정희수",	"조경태",	"조명철",	"조원진",	"조정식",	"조해진",	"조현룡",	"주승용",	"주영순",	"주호영",	"진선미",	"진성준",	"진영",	"천정배",	"최경환",	"최규성",	"최동익",	"최민희",	"최봉홍",	"최원식",	"최재성",	"최재천",	"추미애",	"하태경",	"한기호",	"한명숙",	"한선교",	"한정애",	"함진규",	"홍문종",	"홍문표",	"홍영표",	"홍의락",	"홍익표",	"홍일표",	"홍종학",	"홍지만",	"홍철호",	"황영철",	"황우여",	"황인자",	"황주홍",	"황진하",	"강길부",	"강병원",	"강석진",	"강석호",	"강창일",	"강효상",	"강훈식",	"경대수",	"고용진",	"곽대훈",	"곽상도",	"권미혁",	"권석창",	"권성동",	"권은희",	"권칠승",	"금태섭",	"기동민",	"김경수",	"김경진",	"김경협",	"김관영",	"김광림",	"김광수",	"김규환",	"김기선",	"김도읍",	"김동철",	"김두관",	"김명연",	"김무성",	"김민기",	"김병관",	"김병기",	"김병욱",	"김부겸",	"김삼화",	"김상훈",	"김상희",	"김석기",	"김선동",	"김성수",	"김성식",	"김성원",	"김성찬",	"김성태",	"김성태",	"김성환",	"김세연",	"김수민",	"김순례",	"김승희",	"김영우",	"김영주",	"김영진",	"김영춘",	"김영호",	"김용태",	"김재경",	"김재원",	"김정우",	"김정재",	"김정호",	"김정훈",	"김종대",	"김종민",	"김종석",	"김종인",	"김종회",	"김종훈",	"김중로",	"김진태",	"김진표",	"김철민",	"김태년",	"김태흠",	"김학용",	"김한정",	"김한표",	"김해영",	"김현권",	"김현미",	"김현아",	"나경원",	"남인순",	"노웅래",	"노회찬",	"도종환",	"맹성규",	"문미옥",	"문진국",	"문희상",	"민경욱",	"민병두",	"민홍철",	"박경미",	"박광온",	"박남춘",	"박대출",	"박덕흠",	"박맹우",	"박명재",	"박범계",	"박병석",	"박선숙",	"박성중",	"박순자",	"박영선",	"박완수",	"박완주",	"박용진",	"박인숙",	"박재호",	"박정",	"박주민",	"박주선",	"박주현",	"박준영",	"박지원",	"박찬대",	"박찬우",	"박홍근",	"배덕광",	"백승주",	"백재현",	"백혜련",	"변재일",	"서삼석",	"서영교",	"서청원",	"서형수",	"설훈",	"성일종",	"소병훈",	"손금주",	"손혜원",	"송갑석",	"송기석",	"송기헌",	"송석준",	"송언석",	"송영길",	"송옥주",	"송희경",	"신경민",	"신동근",	"신보라",	"신상진",	"신용현",	"신창현",	"심기준",	"심상정",	"심재권",	"심재철",	"안규백",	"안민석",	"안상수",	"안철수",	"안호영",	"양승조",	"어기구",	"엄용수",	"여상규",	"여영국",	"염동열",	"오세정",	"오신환",	"오영훈",	"오제세",	"우상호",	"우원식",	"원유철",	"원혜영",	"위성곤",	"유기준",	"유동수",	"유민봉",	"유성엽",	"유승민",	"유승희",	"유은혜",	"유의동",	"유재중",	"윤관석",	"윤상직",	"윤상현",	"윤소하",	"윤영석",	"윤영일",	"윤일규",	"윤재옥",	"윤종오",	"윤종필",	"윤준호",	"윤한홍",	"윤호중",	"윤후덕",	"이개호",	"이군현",	"이규희",	"이동섭",	"이만희",	"이명수",	"이상돈",	"이상민",	"이상헌",	"이석현",	"이수혁",	"이양수",	"이언주",	"이완영",	"이용득",	"이용주",	"이용호",	"이우현",	"이원욱",	"이은권",	"이은재",	"이인영",	"이장우",	"이재정",	"이정미",	"이정현",	"이종걸",	"이종구",	"이종명",	"이종배",	"이주영",	"이진복",	"이찬열",	"이채익",	"이철규",	"이철우",	"이철희",	"이춘석",	"이태규",	"이학영",	"이학재",	"이해찬",	"이헌승",	"이현재",	"이혜훈",	"이후삼",	"이훈",	"인재근",	"임이자",	"임재훈",	"임종성",	"장병완",	"장석춘",	"장정숙",	"장제원",	"전재수",	"전해철",	"전현희",	"전혜숙",	"전희경",	"정갑윤",	"정동영",	"정병국",	"정성호",	"정세균",	"정양석",	"정용기",	"정우택",	"정운천",	"정유섭",	"정은혜",	"정인화",	"정재호",	"정점식",	"정종섭",	"정진석",	"정춘숙",	"정태옥",	"제윤경",	"조경태",	"조배숙",	"조승래",	"조원진",	"조응천",	"조정식",	"조훈현",	"주광덕",	"주승용",	"주호영",	"지상욱",	"진선미",	"진영",	"채이배",	"천정배",	"최경환",	"최경환",	"최교일",	"최도자",	"최명길",	"최연혜",	"최운열",	"최인호",	"최재성",	"추경호",	"추미애",	"추혜선",	"표창원",	"하태경",	"한선교",	"한정애",	"함진규",	"허윤정",	"홍문종",	"홍문표",	"홍영표",	"홍의락",	"홍익표",	"홍일표",	"홍철호",	"황영철",	"황주홍",	"황희",
]

In [106]:
len(names)

1899

### 처음 두 단어 중 '위원'이 없는 원소 삭제해주는 코드

In [139]:
def remove_unwanted_elements(cell):
    if isinstance(cell, list):  # cell이 리스트 형인지 확인
        return [s for s in cell if any(word for word in re.split(' |,|\n', s)[:2] if "위원" in word)]
    else:
        return None

# 0번째 열 제외하고 적용
grouped_df.iloc[:, 1:] = grouped_df.iloc[:, 1:].applymap(remove_unwanted_elements)

### names(국회의원 이름 포함된 리스트)에 이름이 없는 원소 삭제해주는 코드

- 몇 개 삭제했는지 확인 가능
- 처음엔 코드가 불안정해서 삭제되어버리는 셀이 있어 추가한 기능인데, 지금은 필요 없음

In [140]:
def remove_elements_without_names_and_count(cell):
    if isinstance(cell, list):  # cell이 리스트 형인지 확인
        original_length = len(cell)
        cell = [s for s in cell if any(word for word in re.split(' |,|\n', s)[:2] if word in names)]
        deleted_elements = original_length - len(cell)
        return cell, deleted_elements
    else:
        return None, 0

# 1. 결과를 저장할 데이터프레임 초기화
deleted_cells_df = pd.DataFrame(index=grouped_df.index, columns=grouped_df.columns[1:], data=0)

# 2. 0번째 열 제외하고 적용
for i in range(1, len(grouped_df.columns)):
    result = grouped_df.iloc[:, i].map(remove_elements_without_names_and_count)
    grouped_df.iloc[:, i] = result.apply(lambda x: x[0])
    deleted_cells_df.iloc[:, i - 1] = result.apply(lambda x: x[1])

# 3. 각 행에서 삭제된 셀의 총 개수 계산
deleted_cells_df['total_deleted_cells'] = deleted_cells_df.sum(axis=1)

# 4. 결과 출력
print(deleted_cells_df)

         1  total_deleted_cells
0       25                   25
1      124                  124
2       57                   57
3       27                   27
4       93                   93
..     ...                  ...
128  12853                12853
129     93                   93
130    111                  111
131     11                   11
132   2910                 2910

[133 rows x 2 columns]


### 위원장 발언 삭제하는 코드(이하 2줄)

- 바로 아래 코드 : "위원장"이라는 단어가 처음 두 단어에 포함되어있는 원소들을 'word_list' 변수에 할당 word_list를 꼭 확인해보고 안에 이상한게 있지 않은지 검사 후 예외를 설정해야함.
- 그 다음 코드 : 'word_list' 변수에 있는 모든 단어가 처음 두 단어에 포함되어 있으면 삭제

In [157]:
from collections import Counter

# '위원장'이 포함된 원소들의 처음 두 단어를 가져오는 함수
def get_elements_with_word(cell):
    if isinstance(cell, list):  # cell이 리스트 형인지 확인
        return [word for s in cell for word in re.split(' |,|\n', s)[:2] if "위원장" in word]
    else:
        return []

# 단어 빈도를 저장할 Counter 객체 초기화
word_freq = Counter()

# 0번째 열 제외하고 적용
for i in range(1, len(grouped_df.columns)):
    # 각 셀에서 '위원장'이 포함된 원소들의 처음 두 단어를 가져옴
    words = grouped_df.iloc[:, i].map(get_elements_with_word)
    # 각 셀에서 가져온 단어들의 빈도를 계산
    for word in words:
        word_freq.update(word)

word_list = [(word, freq) for word, freq in word_freq.most_common()]

소위원장: 131405
위원장: 125175
위원장대리: 17997
소위원장대리: 3872
방송위원회부위원장: 1489
위원장직무대리: 1032
국가청소년위원장: 834
소위원장직무대리: 612
청소년위원장: 476
위원장직무대행: 329
행정자치위원장대리: 160
교육위원장대리: 155
보건복지위원장대리: 114
건설교통위원장대리: 113
재정경제위원장대리: 111
산업자원위원장대리: 110
정무위원장대리: 106
문화관광위원장대리: 106
농림해양수산위원장대리: 89
환경노동위원장대리: 81
통일외교통상위원장대리: 68
국회운영위원장대리: 65
소위원장직무대행: 59
과학기술정보통신위원장대리: 49
법제사법위원장대리: 48
국방위원장대리: 46
국회개혁특별위원장대리: 45
정치개혁특별위원장대리: 34
남북정상회담추진위원장: 31
2014평창동계올림픽유치위원장: 30
여성가족위원장대리: 19
예산결산특별위원장: 11
예산결산특별위원장대리: 9
수산업협동조합중앙회조합감사위원장: 9
건설교통위원장: 8
여성위원장대리: 7
국회운영위원장: 6
노사정위원장: 6
법제사법위원장: 5
농림해양수산위원장: 5
국방위원장: 4
재정경제위원장: 4
소위위원장: 3
여성가족위원장: 3
소소위원장: 2
2014년평창동계올림픽유치특별위원장대리: 2
환경로동위원장대리: 2
정보위원장대리: 2
정보위원장: 2
방송통신특별위원장대리: 2
정무위원장: 2
통일외교통상위원장: 2
행정자치위원장: 2
교육위원장: 2
과학기술정보통신위원장: 2
문화관광위원장: 2
산업자원위원장: 2
보건복지위원장: 2
환경노동위원장: 2
윤리특별위원장: 2
국제경기대회지원특별위원장대리: 1
헌법재판소재판관후보자를겸하는헌법재판소장(이강국)임명동의에관한인사청문특별위원장대리: 1
2011년대구세계육상선수권대회유치특별위원장대리: 1
2014년인천아시아경기대회유치특별위원장대리: 1
투자활성화및일자리창출을위한특별위원장: 1
0소위원장: 1
소우위원장: 1
대법관(양승태)임명동의에관한인사청문특별위원장대리: 1
대법관(김

In [165]:
def remove_attendance_elements(cell):
    if isinstance(cell, list):  # cell이 리스트 형인지 확인
        return [s for s in cell if not any(word for word in re.split(' |,|\n', s)[:2] if word in word_list)]
    else:
        return None

# 0번째 열 제외하고 적용
grouped_df.iloc[:, 1:] = grouped_df.iloc[:, 1:].applymap(remove_attendance_elements)

### 데이터 프레임 안에 비어있는 셀 정보 확인용

In [190]:
empty_count = grouped_df.iloc[:, 1:].isna().sum()
print(f"설정한 구간에서 각 컬럼별로 비어있는 셀 정보\n\n{empty_count}\n")

empty_rows = grouped_df[grouped_df.iloc[:, 1].isna()]
print(f"설정한 구간에서 로우별로 비어있는 셀 정보\n\n{empty_rows}")

설정한 구간에서 각 컬럼별로 비어있는 셀 정보

1번째 컬럼        0
2번째 컬럼       15
3번째 컬럼       28
4번째 컬럼       41
5번째 컬럼       59
           ... 
483번째 컬럼    132
484번째 컬럼    132
485번째 컬럼    132
486번째 컬럼    132
487번째 컬럼    132
Length: 487, dtype: int64

설정한 구간에서 로우별로 비어있는 셀 정보

Empty DataFrame
Columns: [0, 1번째 컬럼, 2번째 컬럼, 3번째 컬럼, 4번째 컬럼, 5번째 컬럼, 6번째 컬럼, 7번째 컬럼, 8번째 컬럼, 9번째 컬럼, 10번째 컬럼, 11번째 컬럼, 12번째 컬럼, 13번째 컬럼, 14번째 컬럼, 15번째 컬럼, 16번째 컬럼, 17번째 컬럼, 18번째 컬럼, 19번째 컬럼, 20번째 컬럼, 21번째 컬럼, 22번째 컬럼, 23번째 컬럼, 24번째 컬럼, 25번째 컬럼, 26번째 컬럼, 27번째 컬럼, 28번째 컬럼, 29번째 컬럼, 30번째 컬럼, 31번째 컬럼, 32번째 컬럼, 33번째 컬럼, 34번째 컬럼, 35번째 컬럼, 36번째 컬럼, 37번째 컬럼, 38번째 컬럼, 39번째 컬럼, 40번째 컬럼, 41번째 컬럼, 42번째 컬럼, 43번째 컬럼, 44번째 컬럼, 45번째 컬럼, 46번째 컬럼, 47번째 컬럼, 48번째 컬럼, 49번째 컬럼, 50번째 컬럼, 51번째 컬럼, 52번째 컬럼, 53번째 컬럼, 54번째 컬럼, 55번째 컬럼, 56번째 컬럼, 57번째 컬럼, 58번째 컬럼, 59번째 컬럼, 60번째 컬럼, 61번째 컬럼, 62번째 컬럼, 63번째 컬럼, 64번째 컬럼, 65번째 컬럼, 66번째 컬럼, 67번째 컬럼, 68번째 컬럼, 69번째 컬럼, 70번째 컬럼, 71번째 컬럼, 72번째 컬럼, 73번째 컬럼, 74번째 컬럼, 75번째 컬럼, 76번째 컬럼, 77번째 컬럼, 78번째 컬럼, 79번째 컬럼, 80번째 컬럼, 81번째 

### 데이터 프레임의 구간을 설정하고 행별로 합치는 코드 - csv 만들기 편하게 구조 변경

In [155]:
grouped_df2 = grouped_df.copy()

In [156]:
# 0번째 컬럼 제외하고 모든 셀을 행별로 합치기
grouped_df2['merged'] = grouped_df2.iloc[:, 1:].apply(lambda row: [item for sublist in row if sublist is not None for item in sublist], axis=1)

# 이후 csv 파일로 저장하기 편한 구조로 만들기
final_df = grouped_df2[[0, 'merged']].rename(columns={"0": "Index", "merged": "Content"})

In [148]:
final_df.iloc[0,1]

['위원장직무대행 박찬석 그러면 회의를 주재토록 하겠습니다. 국회법에 따라 위원장이 선임될 때까지 본 위원이 연장자로서 위원장 직무를 대행하게 되었습니다. 우리 특별위원회의 위원장으로 덕망 있고 훌륭한 분이 선임되기를 기대하면서 오늘 의사일정을 진행하도록 하겠습니다. 성원이 되었으므로 제264회 국회 제1차 2011년대구세계육상선수권대회유치특별위원회를 개의토록 하겠습니다.1. 위원장 선임의 건',
 '위원장직무대행 박찬석 의사일정 제1항 위원장 선임의 건을 상정토록 하겠습니다. 특별위원회의 위원장 선임은 국회법 제47조제1항에 의거 위원회에서 호선하고 본회의에 보고토록 되어 있습니다. 지금까지의 관례상 특별위원회의 위원장은 구두호천에 의하여 선임되어 온 것으로 알고 있습니다. 위원장 선임 방법에 관하여 위원님들께서 다른 의견이 있으시면 말씀하여 주시기를 바랍니다. (｢없습니다｣ 하는 위원 있음) 의견이 없으시면 관례에 따라서 구두호천에 의하여 선임하도록 하고자 하는데 이의가 없으십니까? (｢없습니다｣ 하는 위원 있음) 가결되었음을 선포합니다. 그러면 위원장 선임 방법이 구두호천으로 하기로 결정되었으므로 위원님들께서 위원장님으로 선임하여 주실 분을 추천해 주시기 바랍니다.',
 '곽성문 위원 위원장님!',
 '위원장직무대행 박찬석 곽성문 위원님 말씀해 주십시오.',
 '곽성문 위원 대구 중․남구 출신 곽성문 위원입니다." 육상선수권대회유치특별위원회 위원 중에 최다선이신 3선의 존경하는 박종근 위원이 계시고  또 박 위원께서 재경위원장과 예결위원회 위원장 등을 역임하시면서 여러 가지 지도력과 경륜을 탁월하게 보여 주셨습니다." 또 마침 개최지인 대구지역 출신 위원으로서 개최를 추진하고 있는 대구시라든가 유치위원회와의 업무 협조라든가 이런 여러 가지 점에서 원활하게 업무를 수행할 수 있기 때문에 가장 적임자라 생각을 합니다. 아마 이 점에는 여러 위원님들도 동의를 하시리라 생각합니다. 그래서 본 위원은 존경하는 박종근 위원님을 우리 특위의 위원장으로 추천하고자 합니

In [174]:
# final_df를 csv로 저장
final_df.to_csv('final.csv', index=False)

### 17대 국회의원 이름

- names_R_17 = 17대 보수당 국회의원
- names_L_17 = 17대 민주당 국회의원

In [149]:
names_17 = ["강기갑",	"강기정",	"강길부",	"강봉균",	"강성종",	"강재섭",	"강창일",	"강혜숙",	"고경화",	"고조흥",	"고진화",	"고흥길",	"고희선",	"공성진",	"곽성문",	"구논회",	"권경석",	"권선택",	"권영길",	"권영세",	"권오을",	"권철현",	"김광원",	"김교흥",	"김근태",	"김기석",	"김기춘",	"김기현",	"김낙성",	"김낙순",	"김덕규",	"김덕룡",	"김동철",	"김맹곤",	"김명자",	"김명주",	"김무성",	"김문수",	"김병호",	"김부겸",	"김석준",	"김선미",	"김성곤",	"김성조",	"김송자",	"김애실",	"김양수",	"김영대",	"김영덕",	"김영선",	"김영숙",	"김영주",	"김영춘",	"김용갑",	"김우남",	"김원기",	"김원웅",	"김재경",	"김재원",	"김재윤",	"김재홍",	"김정권",	"김정부",	"김정훈",	"김종률",	"김종인",	"김진표",	"김춘진",	"김충환",	"김태년",	"김태홍",	"김태환",	"김학송",	"김학원",	"김한길",	"김혁규",	"김현미",	"김형오",	"김형주",	"김홍업",	"김홍일",	"김효석",	"김희선",	"김희정",	"나경원",	"남경필",	"노영민",	"노웅래",	"노현송",	"노회찬",	"단병호",	"류근찬",	"맹형규",	"문병호",	"문석호",	"문학진",	"문희",	"문희상",	"민병두",	"박계동",	"박근혜",	"박기춘",	"박명광",	"박병석",	"박상돈",	"박성범",	"박세일",	"박세환",	"박순자",	"박승환",	"박영선",	"박재완",	"박종근",	"박진",	"박찬석",	"박찬숙",	"박창달",	"박혁규",	"박형준",	"박홍수",	"박희태",	"배기선",	"배일도",	"백원우",	"변재일",	"복기왕",	"서갑원",	"서병수",	"서상기",	"서재관",	"서혜석",	"선병렬",	"손봉숙",	"송영길",	"송영선",	"신계륜",	"신국환",	"신기남",	"신명",	"신상진",	"신중식",	"신학용",	"심대평",	"심상정",	"심재덕",	"심재엽",	"심재철",	"안경률",	"안명옥",	"안민석",	"안병엽",	"안상수",	"안영근",	"안택수",	"안홍준",	"양승조",	"양형일",	"엄호성",	"염동연",	"오시덕",	"오영식",	"오제세",	"우상호",	"우원식",	"우윤근",	"우제창",	"우제항",	"원혜영",	"원희룡",	"유기준",	"유기홍",	"유선호",	"유승민",	"유승희",	"유시민",	"유인태",	"유재건",	"유정복",	"유필우",	"윤건영",	"윤두환",	"윤원호",	"윤호중",	"이강두",	"이강래",	"이경숙",	"이경재",	"이계경",	"이계안",	"이계진",	"이광재",	"이광철",	"이군현",	"이규택",	"이근식",	"이기우",	"이낙연",	"이덕모",	"이명규",	"이목희",	"이미경",	"이방호",	"이병석",	"이상경",	"이상득",	"이상락",	"이상민",	"이상배",	"이상열",	"이석현",	"이성구",	"이성권",	"이승희",	"이시종",	"이영순",	"이영호",	"이용희",	"이원복",	"이원영",	"이윤성",	"이은영",	"이인기",	"이인영",	"이인제",	"이재오",	"이재웅",	"이재창",	"이정일",	"이종걸",	"이종구",	"이주영",	"이주호",	"이진구",	"이철우",	"이한구",	"이해봉",	"이해찬",	"이혜훈",	"이호웅",	"이화영",	"임인배",	"임종석",	"임종인",	"임채정",	"임태희",	"임해규",	"장경수",	"장복심",	"장영달",	"장윤석",	"장향숙",	"전병헌",	"전여옥",	"전재희",	"정갑윤",	"정덕구",	"정동채",	"정두언",	"정몽준",	"정문헌",	"정병국",	"정봉주",	"정성호",	"정세균",	"정의용",	"정의화",	"정장선",	"정종복",	"정진석",	"정진섭",	"정청래",	"정형근",	"정화원",	"정희수",	"제종길",	"조경태",	"조배숙",	"조성래",	"조성태",	"조순형",	"조승수",	"조일현",	"조정식",	"주성영",	"주승용",	"주호영",	"지병문",	"진수희",	"진영",	"차명진",	"채수찬",	"채일병",	"천영세",	"천정배",	"최경환",	"최구식",	"최규성",	"최규식",	"최병국",	"최성",	"최순영",	"최연희",	"최용규",	"최인기",	"최재성",	"최재천",	"최철국",	"한광원",	"한명숙",	"한병도",	"한선교",	"한화갑",	"허천",	"허태열",	"현애자",	"홍문표",	"홍미영",	"홍재형",	"홍준표",	"홍창선",	"황우여",	"황진하"]
names_R_17 = ["강재섭", 	"고경화", 	"고조흥", 	"고진화", 	"고흥길", 	"고희선", 	"공성진", 	"곽성문", 	"권경석", 	"권영세", 	"권오을", 	"권철현", 	"김광원", 	"김기춘", 	"김기현", 	"김낙성", 	"김덕룡", 	"김명주", 	"김무성", 	"김문수", 	"김병호", 	"김석준", 	"김성조", 	"김애실", 	"김양수", 	"김영덕", 	"김영선", 	"김영숙", 	"김용갑", 	"김재경", 	"김재원", 	"김정권", 	"김정부", 	"김정훈", 	"김충환", 	"김태환", 	"김학송", 	"김학원", 	"김형오", 	"김희정", 	"나경원", 	"남경필", 	"류근찬", 	"맹형규", 	"문희", 	"박계동", 	"박근혜", 	"박성범", 	"박세일", 	"박세환", 	"박순자", 	"박승환", 	"박재완", 	"박종근", 	"박진", 	"박찬숙", 	"박창달", 	"박혁규", 	"박형준", 	"박희태", 	"배일도", 	"서병수", 	"서상기", 	"송영선", 	"신상진", 	"심재엽", 	"심재철", 	"안경률", 	"안명옥", 	"안상수", 	"안택수", 	"안홍준", 	"엄호성", 	"원희룡", 	"유기준", 	"유승민", 	"유정복", 	"윤건영", 	"윤두환", 	"이강두", 	"이경재", 	"이계경", 	"이계진", 	"이군현", 	"이규택", 	"이덕모", 	"이명규", 	"이방호", 	"이병석", 	"이상득", 	"이상배", 	"이성구", 	"이성권", 	"이원복", 	"이윤성", 	"이인기", 	"이인제", 	"이재오", 	"이재웅", 	"이재창", 	"이종구", 	"이주영", 	"이주호", 	"이진구", 	"이한구", 	"이해봉", 	"이혜훈", 	"임인배", 	"임태희", 	"임해규", 	"장윤석", 	"전여옥", 	"전재희", 	"정갑윤", 	"정두언", 	"정문헌", 	"정병국", 	"정의화", 	"정종복", 	"정진섭", 	"정형근", 	"정화원", 	"정희수", 	"주성영", 	"주호영", 	"진수희", 	"진영", 	"차명진", 	"최경환", 	"최구식", 	"최병국", 	"최연희", 	"한선교", 	"허천", 	"허태열", 	"홍문표", 	"홍준표", 	"황우여", 	"황진하" ]
names_L_17 = ["강기정",	"강길부",	"강봉균",	"강성종",	"강창일",	"강혜숙",	"구논회",	"권선택",	"김교흥",	"김근태",	"김기석",	"김낙순",	"김덕규",	"김동철",	"김맹곤",	"김명자",	"김부겸",	"김선미",	"김성곤",	"김송자",	"김영주",	"김영춘",	"김우남",	"김원기",	"김원웅",	"김재윤",	"김재홍",	"김종률",	"김진표",	"김춘진",	"김태년",	"김태홍",	"김한길",	"김혁규",	"김현미",	"김형주",	"김홍업",	"김희선",	"노영민",	"노웅래",	"노현송",	"문병호",	"문석호",	"문학진",	"문희상",	"민병두",	"박기춘",	"박명광",	"박병석",	"박상돈",	"박영선",	"박찬석",	"박홍수",	"배기선",	"백원우",	"변재일",	"복기왕",	"서갑원",	"서재관",	"서혜석",	"선병렬",	"송영길",	"신계륜",	"신기남",	"신명",	"신중식",	"신학용",	"심재덕",	"안민석",	"안병엽",	"안영근",	"양승조",	"양형일",	"염동연",	"오시덕",	"오영식",	"오제세",	"우상호",	"우원식",	"우윤근",	"우제창",	"우제항",	"원혜영",	"유기홍",	"유선호",	"유승희",	"유시민",	"유인태",	"유재건",	"유필우",	"윤원호",	"윤호중",	"이강래",	"이경숙",	"이계안",	"이광재",	"이광철",	"이근식",	"이기우",	"이목희",	"이미경",	"이상경",	"이상락",	"이상민",	"이석현",	"이시종",	"이영호",	"이용희",	"이원영",	"이은영",	"이인영",	"이종걸",	"이철우",	"이해찬",	"이호웅",	"이화영",	"임종석",	"임종인",	"임채정",	"장경수",	"장복심",	"장영달",	"장향숙",	"전병헌",	"정덕구",	"정동채",	"정봉주",	"정성호",	"정세균",	"정의용",	"정장선",	"정청래",	"제종길",	"조경태",	"조배숙",	"조성래",	"조성태",	"조순형",	"조일현",	"조정식",	"주승용",	"지병문",	"채수찬",	"채일병",	"천정배",	"최규성",	"최규식",	"최성",	"최용규",	"최재성",	"최재천",	"최철국",	"한광원",	"한명숙",	"한병도",	"홍미영",	"홍재형",	"홍창선"]

In [157]:
def check_and_replace(element):
    words = element.split()
    first_two_words = ' '.join(words[:2])
    
    if any(name in first_two_words for name in names_R_17):
        return '1 ' + ' '.join(words[2:])
    elif any(name in first_two_words for name in names_L_17):
        return '0 ' + ' '.join(words[2:])
    else:
        return element  # If no match is found, return the original element

final_df['Content'] = final_df['Content'].apply(lambda row: [check_and_replace(item) for item in row])

In [162]:
final_df

,0,Content
0,2011년대구세계육상선수권대회유치특별위원회,[0 그러면 회의를 주재토록 하겠습니다. 국회법에 따라 위원장이 선임될 때까지 본 ...
1,2012년여수세계박람회유치특별위원회,[0 좌석을 정돈해 주시기 바랍니다. 성원이 되었으므로 제3차 2012년여수세계박람...
2,2014년인천아시아경기대회유치특별위원회,[0 의석을 정돈해 주시기 바랍니다. 성원이 되었으므로 제266회 국회(임시회) 제...
3,2014년평창동계올림픽유치특별위원회,[1 의석을 정돈해 주시기 바랍니다. 아시다시피 정덕구 위원장께서 의원직을 사퇴함에...
4,GP총기사고진상조사소위원회,[0 성원이 되었으므로 제254회 국회 국방위원회 제3차 GP총기사고진상조사소위원회...
...,...,...
128,행정자치위원회,[0 좌석을 정돈해 주시기 바랍니다. 성원이 되었으므로 제258회 국회(임시회․폐회...
129,헌법재판소장(전효숙)임명동의및헌법재판소재판관(목영준.이동흡)선출에관한인사청문특별위원회,[0 성원이 되었으므로 제2차 헌법재판소장(전효숙)임명동의및헌법재판소재판관(목영준․...
130,헌법재판소재판관(조대현)선출에관한인사청문특별위원회,[0 열린우리당 서울 강동을 출신 이상경 위원입니다. 최근 들어 몇 가지 중대한 헌...
131,헌법재판소재판관후보자를겸한헌법재판소장(이강국)임명동의에관한인사청문특별위원회,[1 의석을 정돈해 주시기 바랍니다. 성원이 되었으므로 제3차 헌법재판소재판관후보자...


In [164]:
# Step 1: Concatenate all items in the 'Content' lists into a new list 'the_contents'
the_contents = []
for lst in final_df['Content']:
    the_contents.extend(lst)

# Step 2: The 'the_contents' list now contains all elements of the 'Content' column in 'final_df'
# print(the_contents) to check

# Step 3: Create a DataFrame from 'the_contents'
df_contents = pd.DataFrame(the_contents, columns=['Content'])

In [167]:
the_contents

['0 그러면 회의를 주재토록 하겠습니다. 국회법에 따라 위원장이 선임될 때까지 본 위원이 연장자로서 위원장 직무를 대행하게 되었습니다. 우리 특별위원회의 위원장으로 덕망 있고 훌륭한 분이 선임되기를 기대하면서 오늘 의사일정을 진행하도록 하겠습니다. 성원이 되었으므로 제264회 국회 제1차 2011년대구세계육상선수권대회유치특별위원회를 개의토록 하겠습니다.1. 위원장 선임의 건',
 '0 의사일정 제1항 위원장 선임의 건을 상정토록 하겠습니다. 특별위원회의 위원장 선임은 국회법 제47조제1항에 의거 위원회에서 호선하고 본회의에 보고토록 되어 있습니다. 지금까지의 관례상 특별위원회의 위원장은 구두호천에 의하여 선임되어 온 것으로 알고 있습니다. 위원장 선임 방법에 관하여 위원님들께서 다른 의견이 있으시면 말씀하여 주시기를 바랍니다. (｢없습니다｣ 하는 위원 있음) 의견이 없으시면 관례에 따라서 구두호천에 의하여 선임하도록 하고자 하는데 이의가 없으십니까? (｢없습니다｣ 하는 위원 있음) 가결되었음을 선포합니다. 그러면 위원장 선임 방법이 구두호천으로 하기로 결정되었으므로 위원님들께서 위원장님으로 선임하여 주실 분을 추천해 주시기 바랍니다.',
 '1 위원장님!',
 '0 곽성문 위원님 말씀해 주십시오.',
 '1 대구 중․남구 출신 곽성문 위원입니다." 육상선수권대회유치특별위원회 위원 중에 최다선이신 3선의 존경하는 박종근 위원이 계시고 또 박 위원께서 재경위원장과 예결위원회 위원장 등을 역임하시면서 여러 가지 지도력과 경륜을 탁월하게 보여 주셨습니다." 또 마침 개최지인 대구지역 출신 위원으로서 개최를 추진하고 있는 대구시라든가 유치위원회와의 업무 협조라든가 이런 여러 가지 점에서 원활하게 업무를 수행할 수 있기 때문에 가장 적임자라 생각을 합니다. 아마 이 점에는 여러 위원님들도 동의를 하시리라 생각합니다. 그래서 본 위원은 존경하는 박종근 위원님을 우리 특위의 위원장으로 추천하고자 합니다.',
 '0 박종근 위원님이 추천되었습니다. 또 다른 분을 추천하실 위

In [166]:
df_contents

,Content
0,0 그러면 회의를 주재토록 하겠습니다. 국회법에 따라 위원장이 선임될 때까지 본 위...
1,0 의사일정 제1항 위원장 선임의 건을 상정토록 하겠습니다. 특별위원회의 위원장 선...
2,1 위원장님!
3,0 곽성문 위원님 말씀해 주십시오.
4,"1 대구 중․남구 출신 곽성문 위원입니다."" 육상선수권대회유치특별위원회 위원 중에 ..."
...,...
1330974,1 의결을 해 달라는 얘기입니까?
1330975,1 글쎄 이미 의결은……
1330976,"단병호 위원 알겠습니다. """
1330977,1 예 그리고 검증 의결하기 이전에 우리가 자료제출 요구한 것만으로도 이미 벌써 안...


In [169]:
# Split the 'Content' column based on the first space and expand into new dataframe
split_df = df_contents['Content'].str.split(' ', n=1, expand=True)

# Assign the first split to 'L|R' column and second split to 'Content' column
df_contents['L|R'] = split_df[0]
df_contents['Content'] = split_df[1]

In [170]:
df_contents

,Content,L|R
0,그러면 회의를 주재토록 하겠습니다. 국회법에 따라 위원장이 선임될 때까지 본 위원이...,0
1,의사일정 제1항 위원장 선임의 건을 상정토록 하겠습니다. 특별위원회의 위원장 선임은...,0
2,위원장님!,1
3,곽성문 위원님 말씀해 주십시오.,0
4,"대구 중․남구 출신 곽성문 위원입니다."" 육상선수권대회유치특별위원회 위원 중에 최다...",1
...,...,...
1330974,의결을 해 달라는 얘기입니까?,1
1330975,글쎄 이미 의결은……,1
1330976,"위원 알겠습니다. """,단병호
1330977,예 그리고 검증 의결하기 이전에 우리가 자료제출 요구한 것만으로도 이미 벌써 안 하...,1


In [176]:
# Getting the values and their frequencies in 'L|R' column
print(df_contents['L|R'].value_counts())

L|R
0    646188
1    596979
Name: count, dtype: int64


In [174]:
# Keeping only the rows where 'L|R' is '0' or '1'
df_contents = df_contents[df_contents['L|R'].isin(['0', '1'])]

In [177]:
df_contents = df_contents.reindex(columns=['L|R', 'Content'])

In [179]:
df_contents = df_contents.reset_index(drop=True)

In [180]:
df_contents

,L|R,Content
0,0,그러면 회의를 주재토록 하겠습니다. 국회법에 따라 위원장이 선임될 때까지 본 위원이...
1,0,의사일정 제1항 위원장 선임의 건을 상정토록 하겠습니다. 특별위원회의 위원장 선임은...
2,1,위원장님!
3,0,곽성문 위원님 말씀해 주십시오.
4,1,"대구 중․남구 출신 곽성문 위원입니다."" 육상선수권대회유치특별위원회 위원 중에 최다..."
...,...,...
1243162,1,다음 회의 때보다도 노사정위원회에서 갖다가 보여 주시든가 하여튼 그런 절차를 밟으시...
1243163,1,의결을 해 달라는 얘기입니까?
1243164,1,글쎄 이미 의결은……
1243165,1,예 그리고 검증 의결하기 이전에 우리가 자료제출 요구한 것만으로도 이미 벌써 안 하...


In [182]:
file_path = 'D:\MJ 데이터 공모전 로우 데이터\\17대 민주당_보수당 구분.csv'
df_contents.to_csv(file_path, index=False)

In [172]:
unique_data_list

['0',
 '1',
 '강기갑',
 '대한올림픽위원회총무',
 '천영세',
 '2014평창동계올림픽유치위원장',
 '정진석',
 '손봉숙',
 '이상열',
 '이영순',
 '최인기',
 '이낙연',
 '위원장대리',
 '중앙인사위원회사무처장',
 '금융감독위원회상임위원',
 '국가청소년위원회사무처장',
 '노회찬',
 '청가위원(1인)',
 '한화갑',
 '정몽준',
 '김효석',
 '최순영',
 '전문위원',
 '단병호',
 '김종인',
 '현애자',
 '김홍일',
 '이정일',
 '위원장',
 '소위원장',
 '조승수',
 '위원장직무대행',
 '산업자원위원장대리',
 '보건복지위원장대리',
 '환경노동위원장대리',
 '교육위원장대리',
 '재정경제위원장대리',
 '건설교통위원장대리',
 '행정자치위원장대리',
 '문화관광위원장대리',
 '국회운영위원장',
 '통일외교통상위원장대리',
 '법제사법위원장대리',
 '여성가족위원장대리',
 '심상정',
 '중앙인사위원회중앙공무원교육원장',
 '권영길',
 '재정경제부공적자금관리위원회사무국장',
 '공적자금관리위원회사무국장',
 '수산업협동조합중앙회조합감사위원장',
 '방송위원회상임위원',
 '방송위원회부위원장',
 '방송위원회부위장',
 '방송위원회디지털방송활성화추진단부단장',
 '대한체육회대한올림픽위원회총무',
 '방송위원회방송정책실기술정책부장',
 '방송위원장부위원장',
 '이승희',
 '소위원장대리',
 '전국택시노동조합연맹위원장',
 '중앙인사위원회사무처창',
 '수석전문위원',
 '청소년위원장',
 '청소년위원회사무처장',
 '부패방지위원회사무처장',
 '국가청소년위원장',
 '농림해양수산위원장대리',
 '정치개혁특별위원장대리',
 '국가청렴위원회사무처장',
 '김영대',
 '신국환',
 '신행정수도후속대책위원회기획단장',
 '심대평',
 '금융감독위원회감독정책1국장',
 '금융감독위원회감사정책1국장',
 '금융통화위원',
 '소위원장직무대행',
 '금융감독위원회감독정책2국장',
 '금융감독위원회증권

### dt_1과 dt_0으로 나누어 content만 저장하는 코드

In [275]:
dt_1 = [item for sublist in final_df['Content'] for item in sublist if item.startswith('1 ')]
dt_0 = [item for sublist in final_df['Content'] for item in sublist if item.startswith('2 ')]

In [163]:
the_content = [[item for sublist in final_df['Content'] for item in sublist if item.startswith('1 '|'0 ')]]

TypeError: unsupported operand type(s) for |: 'str' and 'str'

In [276]:
import sys

print("Memory size of dt_1 in bytes: ", sys.getsizeof(dt_1))
print("Memory size of dt_0 in bytes: ", sys.getsizeof(dt_0))

Memory size of dt_1 in bytes:  12029656
Memory size of dt_0 in bytes:  12029656


In [279]:
print(len(dt_1))
print(len(dt_0))

1454100
1432023


In [283]:
dt_1

['위원장님!',
 '대구 중․남구 출신 곽성문 위원입니다." 육상선수권대회유치특별위원회 위원 중에 최다선이신 3선의 존경하는 박종근 위원이 계시고 또 박 위원께서 재경위원장과 예결위원회 위원장 등을 역임하시면서 여러 가지 지도력과 경륜을 탁월하게 보여 주셨습니다." 또 마침 개최지인 대구지역 출신 위원으로서 개최를 추진하고 있는 대구시라든가 유치위원회와의 업무 협조라든가 이런 여러 가지 점에서 원활하게 업무를 수행할 수 있기 때문에 가장 적임자라 생각을 합니다. 아마 이 점에는 여러 위원님들도 동의를 하시리라 생각합니다. 그래서 본 위원은 존경하는 박종근 위원님을 우리 특위의 위원장으로 추천하고자 합니다.',
 '곽성문 위원입니다." 저는 대구 지역 출신이니까 이 대회를 유치해야 되겠습니다마는 우리 여당 열린우리당의 대구사랑의원모임의 특히 광주의 지병문 위원님을 비롯한 여러 위원님들이 이렇게 전폭적으로 참 도와주시고 적극 참여해 주신 점 대단히 감사하게 생각을 합니다." 이런 계기로 대회 유치는 물론이고 여야 간의 관계라든가 동서 화합이라든가 여러 가지 우리 사회의 갈등을 푸는 데도 큰 도움이 되리라고 생각하면서 많이 도와주시기를 다시 한번 부탁드리겠습니다.',
 '경북 영주 출신 장윤석 위원입니다." 세계 3대 스포츠대회가 월드컵대회 하계올림픽대회 그리고 세계육상선수권대회로 알고 있습니다."" 이미 우리나라는 월드컵 하계올림픽을 개최한 스포츠 강국입니다. 2011년에 세계육상선수권대회를 꼭 유치함으로써 대한민국이 스포츠 강국으로서 세계에 떨칠 수 있도록 미력이나마 다하고자 합니다." 감사합니다.',
 '경북 경주 출신 정종복 위원입니다. " 여러분 우선 새해 복 많이 받으십시오. " 국민의 염원이자 국가지대사인 대구세계육상선수권대회가 반드시 유치되도록 존경하는 위원님들과 함께 최선을 다하겠습니다.',
 '곽성문 위원입니다. 세계육상선수권대회유치 지지 결의안의 제안 이유를 간단히 말씀드리겠습니다. " 세계육상선수권대회는 월드컵․하계올림픽과 더불어 세계 3대 

In [284]:
dt_0

['김부겸 위원입니다. 여러 가지로 일정이 빡빡합니다마는 최선을 다해서 저희 국회가 대구에 육상경기대회가 유치될 수 있도록 최선의 노력과 또 협력을 했으면 합니다. 잘 부탁드립니다.',
 '박기춘 위원입니다. 지역구는 경기도 남양주입니다. 평소에 존경하는 김부겸 위원 덕분에 대구사랑의원모임에도 소속되어 있고 남달리 그쪽에 관심이 많아서 자청해서 이 위원회에 들어왔습니다. 작은 힘이나마 보태서 열심히 일하도록 하겠습니다. 고맙습니다.',
 '대구 출신의 박찬석 위원입니다. 2011년 대구세계육상선수권대회 유치를 위해서 최선의 노력을 다하겠습니다. 감사합니다."',
 '광주광역시 남구 출신의 열린우리당 지병문입니다. 선배․동료 위원님들과 함께 대구세계육상선수권대회 유치 활동에 작은 힘이나마 보태겠습니다. 감사합니다.',
 '안녕하십니까? 채일병 위원입니다. 제가 민주당 의원총회 때문에 좀 늦어서 죄송합니다. 저는 전남 해남․진도 출신 민주당 위원입니다. 비교섭단체라 제일 우측의 두 번째에 있네요. 제가 특별히 이 특위를 희망했습니다. 다른 이유라기보다는 제가 존경해 마지않는 김범일 대구시장님이 저하고 30년 공직 동지입니다. 그래서 전라도 사람이 대구의 행사에 열심히 기여하면 동서 화합의 틀이 잡혀지는 것 아니냐 하는 측면에서 제가 앞으로 열심히 할 각오로 여기에 참여했습니다. 위원장님 이하 많은 지도 편달 부탁드립니다. 열심히 하겠습니다.',
 '반갑습니다. 열심히 하겠습니다.',
 '최선을 다하겠습니다.',
 '박찬석 위원입니다." 장관님께 좀 물어보려고 그러는데요 지금 우리나라가 가시적으로 세계적인 체육대회를 유치하려고 하는 것이 몇 건 있습니까?""',
 '그런데 이 3개가 각각 약간의 갈등 관계라든가 혹은 경쟁 관계라든가 이런 것은 없습니까?"',
 '윈-윈 되도록 해야 된다는 장관님 생각은 대단히 높이 평가할 만합니다." 그런데 사실 정부는 지금 평창만 계속 지원하고 경제적 파급효과라든가 대회 규모가 가장 크다고 생각되는 세계육상대회에 대해서는 지원이 

In [282]:
dt_1 = [s.replace('1 ', '', 1) for s in dt_1]
dt_0 = [s.replace('2 ', '', 1) for s in dt_0]

In [285]:
import numpy as np

# 두 리스트의 길이 비교
len_dt_1 = len(dt_1)
len_dt_0 = len(dt_0)

# 더 짧은 리스트에 NaN 값 추가
if len_dt_1 < len_dt_0:
    dt_1 += [np.nan] * (len_dt_0 - len_dt_1)
elif len_dt_0 < len_dt_1:
    dt_0 += [np.nan] * (len_dt_1 - len_dt_0)

In [286]:
# 데이터프레임 생성
L_R_17 = pd.DataFrame({
    '보수': dt_1,
    '진보': dt_0
})

# CSV 파일로 저장
L_R_17.to_csv('17대 보수, 진보 발언 총합.csv', index=False, encoding='utf-8-sig')

### 데이터 프레임 열과 행 바꾸는 코드

In [30]:
# 데이터프레임을 전치 (Transpose)
transposed_df = grouped_df2.transpose()

# 새로운 열 이름 설정
new_header = transposed_df.iloc[0] 

# 첫 번째 행을 삭제하고 새로운 헤더를 설정
transposed_df = transposed_df[1:] 
transposed_df.columns = new_header

### 사람 이름 추출하는 코드

In [ ]:
# 문자열에서 처음 두 단어 추출
new_df['first_two_words'] = new_df['value'].str.split().str[0:2].str.join(' ')

# 처음 두 단어 삭제
new_df['value'] = new_df['value'].apply(lambda x: ' '.join(x.split(' ')[2:]))

### csv 파일 저장

In [139]:
file_path = 'D:\MJ 데이터 공모전 로우 데이터\\deleted_cells_df.csv'
deleted_cells_df.to_csv(file_path, index=False)

---
---
# 국회의원 명단 전처리
---
---

In [240]:
# CSV 파일 경로
file_path = 'D:\\MJ 데이터 공모전 로우 데이터\\국회의원 명단\\15대 이후 국회의원 총 명단.csv'

# CSV 파일 불러오기
C_members = pd.read_csv(file_path, encoding='utf-8', header=0)

In [241]:
# 정규 표현식 패턴: 괄호 안의 모든 문자를 찾습니다.
pattern = "\((.*?)\)"

# '대별 및 소속정당(단체)' 컬럼에 대하여, 패턴에 맞는 문자열을 찾아서 새로운 컬럼으로 추가합니다.
C_members['지역구'] = C_members['대별 및 소속정당(단체)'].apply(lambda x: re.findall(pattern, x))

# '대별 및 소속정당(단체)' 컬럼에서 패턴에 맞는 문자열을 제거합니다.
C_members['대별 및 지역'] = C_members['대별 및 소속정당(단체)'].apply(lambda x: re.sub(pattern, '', x))

In [242]:
C_members

,대별,대별 및 소속정당(단체),이름,이름(한자),생년월일,지역구,대별 및 지역
0,15대의원,제14대국회의원(전국구) 민자당 제15대국회의원(전국구) 신한국당,이명박,李明博,1941-12-19,"[전국구, 전국구]",제14대국회의원 민자당 제15대국회의원 신한국당
1,15대의원,제12대국회의원(부산 부산진) 한국국민당 제14대국회의원(부산 동래갑) 민자당 제1...,강경식,姜慶植,1936-05-10,"[부산 부산진, 부산 동래갑, 부산 동래을]",제12대국회의원 한국국민당 제14대국회의원 민자당 제15대국회의원 신한국당
2,15대의원,제12대 국회의원(마산시)신한민주당 제13대 국회의원(마산시을)통일민주당 제14대 ...,강삼재,姜三載,1952-07-01,"[마산시, 마산시을, 마산회원구, 마산회원구, 마산회원구]",제12대 국회의원신한민주당 제13대 국회의원통일민주당 제14대 국회의원민주자유당 제...
3,15대의원,제15대 국회의원(성북을)한나라당,강성재,姜聲才,1939-03-11,[성북을],제15대 국회의원한나라당
4,15대의원,제12대 국회의원(전국구) 민주정의당 제14대 국회의원(전국구) 민자당 제15대 국...,강용식,康容植,1939-05-08,"[전국구, 전국구, 전국구]",제12대 국회의원 민주정의당 제14대 국회의원 민자당 제15대 국회의원 신한국당
...,...,...,...,...,...,...,...
1894,20대의원,제18대국회의원(인천 남구갑) 한나라당 제19대국회의원(인천 남구갑) 새누리당 제2...,홍일표,洪日杓,1956-02-11,"[인천 남구갑, 인천 남구갑, 인천 남구갑]",제18대국회의원 한나라당 제19대국회의원 새누리당 제20대국회의원 새누리당
1895,20대의원,제19대국회의원(경기 김포) 새누리당 제20대국회의원(경기 김포시을) 새누리당,홍철호,洪哲鎬,1958-08-13,"[경기 김포, 경기 김포시을]",제19대국회의원 새누리당 제20대국회의원 새누리당
1896,20대의원,제18대국회의원(강원 홍천·횡성) 한나라당 제19대국회의원(강원 홍천·횡성) 새누리...,황영철,黃永哲,1965-07-13,"[강원 홍천·횡성, 강원 홍천·횡성, 강원 홍천군철원군화천군양구군인제군]",제18대국회의원 한나라당 제19대국회의원 새누리당 제20대국회의원 새누리당
1897,20대의원,제19대국회의원(전남 장흥군강진군영암군) 새정치민주연합 제20대국회의원(전남 고흥군...,황주홍,黃柱洪,1952-02-27,"[전남 장흥군강진군영암군, 전남 고흥군보성군장흥군강진군]",제19대국회의원 새정치민주연합 제20대국회의원 국민의당


In [243]:
# 정규 표현식 패턴: 괄호 안의 모든 문자를 찾습니다.
pattern = "국회의원"

# '대별 및 소속정당(단체)' 컬럼에 대하여, 패턴에 맞는 문자열을 찾아서 새로운 컬럼으로 추가합니다.
C_members['지역구'] = C_members['대별 및 지역'].apply(lambda x: re.findall(pattern, x))

# '대별 및 소속정당(단체)' 컬럼에서 패턴에 맞는 문자열을 제거합니다.
C_members['대별 및 지역'] = C_members['대별 및 소속정당(단체)'].apply(lambda x: re.sub(pattern, '', x))

TypeError: <lambda>() got an unexpected keyword argument 'expend'

In [239]:
C_members

,대별,대별 및 소속정당(단체),이름,이름(한자),생년월일,지역구,대별 및 지역,대별 및 지역1,대별 및 지역2,대별 및 지역3,대별 및 지역4,대별 및 지역5,대별 및 지역6,대별 및 지역7,대별 및 지역8,대별 및 지역9
0,15대의원,제14대국회의원(전국구) 민자당 제15대국회의원(전국구) 신한국당,이명박,李明博,1941-12-19,"[전국구, 전국구]",제14대,민자당 제15대,신한국당,None,None,None,None,None,None,None
1,15대의원,제12대국회의원(부산 부산진) 한국국민당 제14대국회의원(부산 동래갑) 민자당 제1...,강경식,姜慶植,1936-05-10,"[부산 부산진, 부산 동래갑, 부산 동래을]",제12대,한국국민당 제14대,민자당 제15대,신한국당,None,None,None,None,None,None
2,15대의원,제12대 국회의원(마산시)신한민주당 제13대 국회의원(마산시을)통일민주당 제14대 ...,강삼재,姜三載,1952-07-01,"[마산시, 마산시을, 마산회원구, 마산회원구, 마산회원구]",제12대,신한민주당 제13대,통일민주당 제14대,민주자유당 제15대,신한국당 제16대,한나라당,None,None,None,None
3,15대의원,제15대 국회의원(성북을)한나라당,강성재,姜聲才,1939-03-11,[성북을],제15대,한나라당,None,None,None,None,None,None,None,None
4,15대의원,제12대 국회의원(전국구) 민주정의당 제14대 국회의원(전국구) 민자당 제15대 국...,강용식,康容植,1939-05-08,"[전국구, 전국구, 전국구]",제12대,민주정의당 제14대,민자당 제15대,신한국당,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1894,20대의원,제18대국회의원(인천 남구갑) 한나라당 제19대국회의원(인천 남구갑) 새누리당 제2...,홍일표,洪日杓,1956-02-11,"[인천 남구갑, 인천 남구갑, 인천 남구갑]",제18대,한나라당 제19대,새누리당 제20대,새누리당,None,None,None,None,None,None
1895,20대의원,제19대국회의원(경기 김포) 새누리당 제20대국회의원(경기 김포시을) 새누리당,홍철호,洪哲鎬,1958-08-13,"[경기 김포, 경기 김포시을]",제19대,새누리당 제20대,새누리당,None,None,None,None,None,None,None
1896,20대의원,제18대국회의원(강원 홍천·횡성) 한나라당 제19대국회의원(강원 홍천·횡성) 새누리...,황영철,黃永哲,1965-07-13,"[강원 홍천·횡성, 강원 홍천·횡성, 강원 홍천군철원군화천군양구군인제군]",제18대,한나라당 제19대,새누리당 제20대,새누리당,None,None,None,None,None,None
1897,20대의원,제19대국회의원(전남 장흥군강진군영암군) 새정치민주연합 제20대국회의원(전남 고흥군...,황주홍,黃柱洪,1952-02-27,"[전남 장흥군강진군영암군, 전남 고흥군보성군장흥군강진군]",제19대,새정치민주연합 제20대,국민의당,None,None,None,None,None,None,None


In [228]:
# '국회의원'이 몇 번 나오는지 확인하여 새로운 컬럼 생성
split_column = C_members['대별 및 지역1'].str.split(' ', expand=True)
num_occurrences = split_column.shape[1] - 1  # '국회의원'이 나오는 횟수

# '두 번째 컬럼'에 '국회의원' 앞의 단어 할당
C_members['대별 및 지역1'] = split_column[0]

# 새로운 컬럼 생성하여 '국회의원' 뒤의 단어 할당
for i in range(num_occurrences):
    column_name = f'대별 및 지역1-{i+1}'
    C_members[column_name] = split_column[i+1]

# 출력
print(C_members)

         대별                                      대별 및 소속정당(단체)   이름 이름(한자)  \
0     15대의원               제14대국회의원(전국구) 민자당 제15대국회의원(전국구) 신한국당  이명박    李明博   
1     15대의원  제12대국회의원(부산 부산진) 한국국민당 제14대국회의원(부산 동래갑) 민자당 제1...  강경식    姜慶植   
2     15대의원  제12대 국회의원(마산시)신한민주당 제13대 국회의원(마산시을)통일민주당 제14대 ...  강삼재    姜三載   
3     15대의원                                제15대 국회의원(성북을)한나라당   강성재    姜聲才   
4     15대의원  제12대 국회의원(전국구) 민주정의당 제14대 국회의원(전국구) 민자당 제15대 국...  강용식    康容植   
...     ...                                                ...  ...    ...   
1894  20대의원  제18대국회의원(인천 남구갑) 한나라당 제19대국회의원(인천 남구갑) 새누리당 제2...  홍일표    洪日杓   
1895  20대의원      제19대국회의원(경기 김포) 새누리당  제20대국회의원(경기 김포시을) 새누리당   홍철호    洪哲鎬   
1896  20대의원  제18대국회의원(강원 홍천·횡성) 한나라당 제19대국회의원(강원 홍천·횡성) 새누리...  황영철    黃永哲   
1897  20대의원  제19대국회의원(전남 장흥군강진군영암군) 새정치민주연합 제20대국회의원(전남 고흥군...  황주홍    黃柱洪   
1898  20대의원  제20대국회의원(서울 양천구갑) 더불어민주당 제21대국회의원(서울 양천구갑)더불어민주당    황희     黃熙   

            생년월일                                       지역구 대별 및

In [230]:
C_members

,대별,대별 및 소속정당(단체),이름,이름(한자),생년월일,지역구,대별 및 지역,대별 및 지역1,대별 및 지역2,대별 및 지역3,...,대별 및 지역1-1,대별 및 지역1-2,대별 및 지역1-3,대별 및 지역1-4,대별 및 지역1-5,대별 및 지역1-6,대별 및 지역1-7,대별 및 지역1-8,대별 및 지역1-9,대별 및 지역1-10
0,15대의원,제14대국회의원(전국구) 민자당 제15대국회의원(전국구) 신한국당,이명박,李明博,1941-12-19,"[전국구, 전국구]",제14대,,신한국당,None,...,민자당,제15대,None,None,None,None,None,None,None,None
1,15대의원,제12대국회의원(부산 부산진) 한국국민당 제14대국회의원(부산 동래갑) 민자당 제1...,강경식,姜慶植,1936-05-10,"[부산 부산진, 부산 동래갑, 부산 동래을]",제12대,,민자당 제15대,신한국당,...,한국국민당,제14대,None,None,None,None,None,None,None,None
2,15대의원,제12대 국회의원(마산시)신한민주당 제13대 국회의원(마산시을)통일민주당 제14대 ...,강삼재,姜三載,1952-07-01,"[마산시, 마산시을, 마산회원구, 마산회원구, 마산회원구]",제12대,신한민주당,통일민주당 제14대,민주자유당 제15대,...,제13대,,None,None,None,None,None,None,None,None
3,15대의원,제15대 국회의원(성북을)한나라당,강성재,姜聲才,1939-03-11,[성북을],제15대,한나라당,None,None,...,,None,None,None,None,None,None,None,None,None
4,15대의원,제12대 국회의원(전국구) 민주정의당 제14대 국회의원(전국구) 민자당 제15대 국...,강용식,康容植,1939-05-08,"[전국구, 전국구, 전국구]",제12대,,민자당 제15대,신한국당,...,민주정의당,제14대,,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1894,20대의원,제18대국회의원(인천 남구갑) 한나라당 제19대국회의원(인천 남구갑) 새누리당 제2...,홍일표,洪日杓,1956-02-11,"[인천 남구갑, 인천 남구갑, 인천 남구갑]",제18대,,새누리당 제20대,새누리당,...,한나라당,제19대,None,None,None,None,None,None,None,None
1895,20대의원,제19대국회의원(경기 김포) 새누리당 제20대국회의원(경기 김포시을) 새누리당,홍철호,洪哲鎬,1958-08-13,"[경기 김포, 경기 김포시을]",제19대,,새누리당,None,...,새누리당,,제20대,None,None,None,None,None,None,None
1896,20대의원,제18대국회의원(강원 홍천·횡성) 한나라당 제19대국회의원(강원 홍천·횡성) 새누리...,황영철,黃永哲,1965-07-13,"[강원 홍천·횡성, 강원 홍천·횡성, 강원 홍천군철원군화천군양구군인제군]",제18대,,새누리당 제20대,새누리당,...,한나라당,제19대,None,None,None,None,None,None,None,None
1897,20대의원,제19대국회의원(전남 장흥군강진군영암군) 새정치민주연합 제20대국회의원(전남 고흥군...,황주홍,黃柱洪,1952-02-27,"[전남 장흥군강진군영암군, 전남 고흥군보성군장흥군강진군]",제19대,,국민의당,None,...,새정치민주연합,제20대,None,None,None,None,None,None,None,None


In [214]:
import pandas as pd
import random
import string

# 데이터프레임 생성
data = []

# 한글 글자 생성 함수
def generate_hangul(length):
    # 한글 유니코드 시작: 44032, 끝: 55203
    base_code = 44032
    last_code = 55203
    hanguls = []
    for _ in range(length):
        code = random.randint(base_code, last_code)
        hanguls.append(chr(code))
    return ''.join(hanguls)

# 데이터프레임에 랜덤한 한글 글자 추가
for _ in range(10):
    hangul_text = generate_hangul(10)
    data.append({'첫 번째 컬럼': hangul_text, '두 번째 컬럼': hangul_text +'국회의원' + hangul_text})

# 데이터프레임 생성
df = pd.DataFrame(data)

# 출력
print(df)

      첫 번째 컬럼                   두 번째 컬럼
0  혰밊튋꾓븊륙픀톴팳펈  혰밊튋꾓븊륙픀톴팳펈국회의원혰밊튋꾓븊륙픀톴팳펈
1  갩슧떭쭺훴란탈펤봠퇂  갩슧떭쭺훴란탈펤봠퇂국회의원갩슧떭쭺훴란탈펤봠퇂
2  슌즰풼뚹켓퐸쯰쯶뚉귶  슌즰풼뚹켓퐸쯰쯶뚉귶국회의원슌즰풼뚹켓퐸쯰쯶뚉귶
3  섫쟐댞뫨뮹솊굡뇺쉘롱  섫쟐댞뫨뮹솊굡뇺쉘롱국회의원섫쟐댞뫨뮹솊굡뇺쉘롱
4  뎯걄앛왶묢촱펗뫒뼪꼴  뎯걄앛왶묢촱펗뫒뼪꼴국회의원뎯걄앛왶묢촱펗뫒뼪꼴
5  띃햱욋먡룪겺버팃몎뼂  띃햱욋먡룪겺버팃몎뼂국회의원띃햱욋먡룪겺버팃몎뼂
6  뀢푴뛋벪뢬퍺긙쓙럗뒙  뀢푴뛋벪뢬퍺긙쓙럗뒙국회의원뀢푴뛋벪뢬퍺긙쓙럗뒙
7  떤잝왰돴쪣삶쥭먢눬덅  떤잝왰돴쪣삶쥭먢눬덅국회의원떤잝왰돴쪣삶쥭먢눬덅
8  웫뢥압폵퉂렦캈펎히픰  웫뢥압폵퉂렦캈펎히픰국회의원웫뢥압폵퉂렦캈펎히픰
9  헥땦읤젮흞퇐녥뀜쭾뵬  헥땦읤젮흞퇐녥뀜쭾뵬국회의원헥땦읤젮흞퇐녥뀜쭾뵬


In [215]:
# '국회의원'이 몇 번 나오는지 확인하여 새로운 컬럼 생성
split_column = df['두 번째 컬럼'].str.split('국회의원', expand=True)
num_occurrences = split_column.shape[1] - 1  # '국회의원'이 나오는 횟수

# '두 번째 컬럼'에 '국회의원' 앞의 단어 할당
df['두 번째 컬럼'] = split_column[0]

# 새로운 컬럼 생성하여 '국회의원' 뒤의 단어 할당
for i in range(num_occurrences):
    column_name = f'새로운 컬럼{i+1}'
    df[column_name] = split_column[i+1]

# 출력
print(df)

      첫 번째 컬럼     두 번째 컬럼     새로운 컬럼1
0  혰밊튋꾓븊륙픀톴팳펈  혰밊튋꾓븊륙픀톴팳펈  혰밊튋꾓븊륙픀톴팳펈
1  갩슧떭쭺훴란탈펤봠퇂  갩슧떭쭺훴란탈펤봠퇂  갩슧떭쭺훴란탈펤봠퇂
2  슌즰풼뚹켓퐸쯰쯶뚉귶  슌즰풼뚹켓퐸쯰쯶뚉귶  슌즰풼뚹켓퐸쯰쯶뚉귶
3  섫쟐댞뫨뮹솊굡뇺쉘롱  섫쟐댞뫨뮹솊굡뇺쉘롱  섫쟐댞뫨뮹솊굡뇺쉘롱
4  뎯걄앛왶묢촱펗뫒뼪꼴  뎯걄앛왶묢촱펗뫒뼪꼴  뎯걄앛왶묢촱펗뫒뼪꼴
5  띃햱욋먡룪겺버팃몎뼂  띃햱욋먡룪겺버팃몎뼂  띃햱욋먡룪겺버팃몎뼂
6  뀢푴뛋벪뢬퍺긙쓙럗뒙  뀢푴뛋벪뢬퍺긙쓙럗뒙  뀢푴뛋벪뢬퍺긙쓙럗뒙
7  떤잝왰돴쪣삶쥭먢눬덅  떤잝왰돴쪣삶쥭먢눬덅  떤잝왰돴쪣삶쥭먢눬덅
8  웫뢥압폵퉂렦캈펎히픰  웫뢥압폵퉂렦캈펎히픰  웫뢥압폵퉂렦캈펎히픰
9  헥땦읤젮흞퇐녥뀜쭾뵬  헥땦읤젮흞퇐녥뀜쭾뵬  헥땦읤젮흞퇐녥뀜쭾뵬
